In [1]:
!pwd

/dccstor/shanmukh/sravani_internship/benchmark_experiments


# Importing Clients 

In [2]:
from utils.compute_clients import create_clients


In [ ]:
# import pandas as pd
# df = pd.read_csv("benchmark_v4.csv")
# df.info()

In [3]:
df.tail(100)

NameError: name 'df' is not defined

# Combined Instruction Evaluation 

## Prompt Version 1

In [4]:
def build_user_prompt_v1(inst,constraints):
    return f"""Given the following programming instruction and a list of constraints, verify if all constraints are **explicitly mentioned** in the instruction. 

Instruction:
{inst}

Constraints:
{constraints}

Respond with "Yes" if all constraints are included in the instruction. Otherwise, respond with "No".
**Do not infer, assume, or deduce** that a constraint is satisfied unless it is **clearly mentioned **.
If a constraint is only implied or suggested, mark it as **not satisfied**.
Do **not use your own interpretation** or **general knowledge** about what the instruction might mean — judge only what is **actually written**.
output format:

{{
   reason: why is it yes or no if it is no give the list of the constraints missing in the instruction
   aligns: yes/no
   }}
"""


## Prompt Version 2 

In [5]:
import json
from tqdm import tqdm
import pandas as pd



def build_user_prompt_v2(instruction, constraints):
        return f"""You are a verifier. Your task is to evaluate whether a given instruction includes a set of constraints.

You will be provided:
- An instruction
- A list of constraints

Your task:
- Check if each constraint is explicitly mentioned in the instruction.
- **Do not infer, assume, or deduce** that a constraint is satisfied unless it is **clearly mentioned **.
- If a constraint is only implied or suggested, mark it as **not satisfied**.
- Do **not use your own interpretation** or **general knowledge** about what the instruction might mean — judge only what is **actually written**.
- Output your reasoning in a valid JSON format like below:

{{
  "Evaluation": [
    {{
      "Constraint": "<constraint text>",
      "Reason": "explanation",
      "Aligns": [true|false]
    }},
    ...
  ],
  "FinalDecision": {{
    "Score": "<number of constraints met>/<total number of constraints>",
    "Reason": "<Yes if all constraints were satisfied; No if any were violated, with explanation>",
    "Aligns": [true|false]
  }}
}}

Do not include any extra commentary. Return only valid JSON.

[Instruction]:
{instruction}

[Constraints]:
{constraints}
"""
    




## Prompt Version 3 

In [6]:
import json
from tqdm import tqdm
import pandas as pd



def build_user_prompt_v3(instruction, constraints):
        return f"""You are a verifier. Your task is to evaluate whether a given instruction includes a set of constraints.

You will be provided:
- An instruction
- A list of constraints

Your task:
- Check if each constraint is explicitly mentioned in the instruction.
- **Do not infer, assume, or deduce** that a constraint is satisfied unless it is **clearly mentioned **.
- If a constraint is only implied or suggested, mark it as **not satisfied**.
- Do **not use your own interpretation** or **general knowledge** about what the instruction might mean — judge only what is **actually written**.
- Output your reasoning in a valid JSON format like below:

{{
    "FinalDecision": {{
    "Score": "<number of constraints met>/<total number of constraints>",
    "Reason": "<Yes if all constraints were satisfied; No if any were violated, with explanation>",
    "Aligns": [true|false]
  }}
}}

Do not include any extra commentary. Return only valid JSON.

[Instruction]:
{instruction}

[Constraints]:
{constraints}
"""
    




In [ ]:
n = 24
filtered_relevant_constraints = df.iloc[n,9]
combined_instruction = df.iloc[n,15]

print(combined_instruction)
print(filtered_relevant_constraints)

In [ ]:
prompt = build_user_prompt_v1(combined_instruction, filtered_relevant_constraints)
print(prompt)

In [ ]:
wrong_example = f"""
Create a function that takes in two numbers as arguments and returns the product of the two. Additionally, your solution should have a time complexity of O(log n), where n is the larger of the two input numbers. The function should handle edge cases, such as when one or both input numbers are zero, and return the correct product. You should instead implement your own logic to calculate the product using only bitwise operations such as bitwise shift and bitwise AND, as well as basic arithmetic operations such as addition, subtraction, and division. Furthermore, the function should raise an appropriate error if the inputs are not integers. To enhance performance, the implementation should minimize the number of additions performed during the multiplication process. """

In [ ]:
wrong_example_2 = """
Develop an algorithm to find the value of n where n is the index of the Fibonacci sequence. The algorithm must be implemented using a recursive approach, ensuring that it correctly returns the Fibonacci number for all valid inputs, including edge cases like n = 0 and n = 1. Additionally, the algorithm must not use any built-in mathematical functions or libraries for calculating Fibonacci numbers.
"""

## Testing the Prompts

In [7]:
df = pd.read_json("benchmark_dataset/benchmark_v9.jsonl", lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [8]:

import os
import dotenv
dotenv.load_dotenv()
from utils.compute_clients import create_clients
api_key = os.getenv("IBM_OPENAI_API_KEY")
client = create_clients(mode="GPT-azure")
def get_response(n=0,type="v1",wrong=False):
    
    # combined_instruction = df.iloc[n,15]
    if wrong:
        combined_instruction = wrong_example_2
        filtered_relevant_constraints = df.iloc[24,9]
    else:
        combined_instruction = df.iloc[n,15]
        print
        filtered_relevant_constraints = df.iloc[n,9]
    if type == "v1":
        prompt = build_user_prompt_v1(combined_instruction, filtered_relevant_constraints)
    elif type =="v2":
        prompt = build_user_prompt_v2(combined_instruction, filtered_relevant_constraints)
    elif type == "v3":
        prompt = build_user_prompt_v3(combined_instruction, filtered_relevant_constraints)
        
    response = client.get_model_response(user_prompt=prompt)
    return response

In [9]:
import pandas as pd
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v8.jsonl",lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [8]:
filter_df = df[df["dataset"]=="xlangai/DS-1000"]
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 375 to 589
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               215 non-null    int64  
 1   dataset                          215 non-null    object 
 2   instruction                      215 non-null    object 
 3   code                             215 non-null    object 
 4   test                             215 non-null    object 
 5   relevant_categories              215 non-null    object 
 6   simplified_instruction           215 non-null    object 
 7   extracted_constraints            215 non-null    object 
 8   final_comprehensive_constraints  215 non-null    object 
 9   filtered_relevant_constraints    215 non-null    object 
 10  quality_scores                   215 non-null    object 
 11  relevance_score                  215 non-null    float64
 12  objectivity_score        

In [ ]:
wrong_example

In [10]:
get_response(14,"v3")


'{\n  "FinalDecision": {\n    "Score": "8/8",\n    "Reason": "All constraints are explicitly mentioned in the instruction: no use of multiplication operator; no built-in functions for product; time complexity O(log n); use of bitwise operations (shift and AND); use of basic arithmetic operations (addition, subtraction, division); handling edge cases including zero inputs; raising error for non-integer inputs; minimizing additions for performance.",\n    "Aligns": true\n  }\n}'

In [ ]:
get_response(14,"v2",True)

In [ ]:
get_response(14,"v1",True)

In [ ]:
get_response(24,"v1",True)

In [ ]:
get_response(24,"v2",True)

In [ ]:
get_response(24,"v3",True)

In [35]:
import pandas as pd
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v9_v2.jsonl",lines=True)
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

# Classifying the Instruction difficulty 

In [32]:
def difficulty_classification_prompt(instruction):
    return f"""You are a difficulty evaluator for programming tasks.

Your job is to classify a given programming instruction into one of three categories based on its overall difficulty:

- Easy: The task involves basic programming logic, no complex algorithms, and minimal constraints.
- Medium: The task involves little complexity such as use of specific data structures, standard algorithms, or multiple conditions to satisfy.
- Hard: The task involves moderate to high complexity logic, performance optimization, advanced algorithmic thinking, or multiple non-trivial constraints.

Only use the content explicitly mentioned in the instruction. Do not assume extra difficulty unless it is clearly indicated.

Respond in the following JSON format:

{{"Reason": "<Brief explanation for the chosen difficulty>",
  "Difficulty": "<Easy | Medium | Hard>",
  
}}

[Instruction]:
{instruction}
"""


In [3]:
import os
import dotenv
dotenv.load_dotenv()
from utils.compute_clients import create_clients
api_key = os.getenv("IBM_OPENAI_API_KEY")
client = create_clients(mode="GPT-azure")

In [5]:
import re
def extract_first_difficulty(text):
    m = re.search(r'"Difficulty"\s*:\s*"(Easy|Medium|Hard)"', text)
    return m.group(1) if m else None


In [7]:
import re

def extract_difficulty_value(text):
    """
    Use regex to find the first occurrence of "Difficulty": "Easy", "Medium", or "Hard"
    and return it as a string (without quotes), or None if not found.
    """
    matches = re.findall(
        r'"Difficulty"\s*:\s*("(?:Easy|Medium|Hard)")',
        text
    )
    return matches[0].strip('"') if matches else None


In [37]:
id = 364
combined_instruction = df[df["id"]==id]["combined_instruction"].values[0]
# constraints = df.iloc[n,18]
print(combined_instruction)
# print(constraints)
prompt = difficulty_classification_prompt(combined_instruction)
response = client.get_model_response(user_prompt=prompt)
print("Classification:\n",response)

Design a class to represent a point in 2D space. It should store two variables—x and y.

Additionally, implement the following methods:
1. A method to calculate the distance between two points, taking into account the Earth's curvature. Ensure that this method handles any edge cases appropriately and uses efficient algorithms for geometric calculations to minimize computational complexity.
2. A method to calculate the midpoint between two points, considering the elevation difference between the points. This method should also implement input validation to ensure parameters are of the expected types and within valid ranges.
3. A method to check if a given point is inside a given rectangle, considering the rectangle's rotation and scale. Ensure that each method in the class is modular and adheres to the single responsibility principle.
4. A method to check if a given point is inside a given circle, considering the circle's elevation and rotation.
5. A method to rotate the point around th

Classification:
 ```json
{
  "Reason": "The task requires implementing a class with multiple complex geometric methods that consider Earth's curvature, 3D space, elevation, rotation, and scaling. It involves advanced geometric calculations, input validation, modular design principles, and performance optimization, all of which indicate moderate to high complexity.",
  "Difficulty": "Hard"
}
```


In [8]:
extract_first_difficulty(response)

'Easy'

In [9]:
extract_difficulty_value(response)

'Easy'

In [ ]:
import re

def extract_aligns_values(text):
    # Use regex to find all occurrences of "Aligns": true or "Aligns": false
    matches = re.findall(r'"Aligns"\s*:\s*(true|false)', text, re.IGNORECASE)
    # Convert string values to boolean
    return [m.lower() == 'true' for m in matches]

# Example usage
input_text = '''
{
  "Evaluation": [
    {
      "Constraint": "Remove specific directories (e.g., `dist`, `build`, `.eggs`) and their contents.",
      "Reason": "The instruction explicitly states: 'Remove specific directories (e.g., `dist`, `build`, `.eggs`) and their contents, ensuring that all specified directories are thoroughly cleaned.'",
      "Aligns": true
    },
    {
      "Constraint": "Recursively remove directories and files matching certain patterns (e.g., `__pycache__`, `*.pyc`).",
      "Reason": "The instruction explicitly states: 'Recursively remove directories and files matching certain patterns (e.g., `__pycache__`, `*.pyc`), allowing for comprehensive cleanup of unwanted files.'",
      "Aligns": true
    },
    {
      "Constraint": "Remove files matching patterns in specific directories (e.g., `*.png` and `*.yml` files in `docs/test_to_docs`).",
      "Reason": "The instruction explicitly states: 'Remove files matching patterns in specific directories (e.g., `*.png` and `*.yml` files in `docs/test_to_docs`), targeting specific file types for removal.'",
      "Aligns": true
    },
    {
      "Constraint": \"Provide a dry-run option that lists the files and directories that would be removed without actually deleting them.\",
      "Reason": "The instruction explicitly states: 'Provide a dry-run option that lists the files and directories that would be removed without actually deleting them, giving users a preview of the cleanup process.'",
      "Aligns": true
    },
    {
      "Constraint": "Print the actions it is taking (e.g., 'Removing: dist/').",
      "Reason": "The instruction explicitly states: 'Print the actions it is taking (e.g., \"Removing: dist/\")', enhancing the script's documentation and readability.",
      "Aligns": true
    }
  ]
}
'''

aligns = extract_aligns_values(input_text)
print(aligns)


In [ ]:
pwd

# Verifying the Combined Instruction for benchmark_v9 

In [18]:
import pandas as pd

# Correct file path and use lines=True for JSONL format
df = pd.read_json("benchmark_dataset/benchmark_v9.jsonl", lines=True)

# Display basic info about the DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [ ]:
df["filtered_relevant_constraints"][0]

In [13]:
presence_raw = df["constraint_wise_presence"][500]
print(df["constraint_wise_presence"][500])
sum(df["constraint_wise_presence"][500])
presence = [p for p in presence_raw if p]
print(presence)

[True, True, True, True, True]
[True, True, True, True, True]


In [33]:
filtered_df = df[df["dataset"]=="xlangai/DS-1000"]
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 375 to 589
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               215 non-null    int64  
 1   dataset                          215 non-null    object 
 2   instruction                      215 non-null    object 
 3   code                             215 non-null    object 
 4   test                             215 non-null    object 
 5   relevant_categories              215 non-null    object 
 6   simplified_instruction           215 non-null    object 
 7   extracted_constraints            215 non-null    object 
 8   final_comprehensive_constraints  215 non-null    object 
 9   filtered_relevant_constraints    215 non-null    object 
 10  quality_scores                   215 non-null    object 
 11  relevance_score                  215 non-null    float64
 12  objectivity_score        

In [34]:
abnormal_ds_1000 = filtered_df[[len(x)<len(y) for x,y in zip(filtered_df["combined_instruction"], filtered_df["instruction"])]]
abnormal_ds_1000.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 376 to 586
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               10 non-null     int64  
 1   dataset                          10 non-null     object 
 2   instruction                      10 non-null     object 
 3   code                             10 non-null     object 
 4   test                             10 non-null     object 
 5   relevant_categories              10 non-null     object 
 6   simplified_instruction           10 non-null     object 
 7   extracted_constraints            10 non-null     object 
 8   final_comprehensive_constraints  10 non-null     object 
 9   filtered_relevant_constraints    10 non-null     object 
 10  quality_scores                   10 non-null     object 
 11  relevance_score                  10 non-null     float64
 12  objectivity_score         

In [46]:
lengths = [(len(row["combined_instruction"]), len(row["instruction"])) for row in abnormal_ds_1000.to_dict(orient='records')]
print(lengths)

[(1192, 1255), (1794, 1825), (858, 860), (1528, 1533), (1216, 1218), (1692, 1709), (990, 1124), (1442, 1449), (1688, 1695), (933, 936)]


In [35]:
abnormal_ds_1000.to_json("response_outputs/testing_responses/ds_1000_abnormal_combined_instruct_length.jsonl", orient="records", lines=True)

In [25]:
stats = df["constraint_wise_presence"].to_list()
print(len(stats))
stat_final =[]
for stat in stats:
    stat_final.extend(stat)
print(sum(stat_final)/len(stat_final))
print(sum(stat_final))
print(len(stat_final)-sum(stat_final))
print(len(stat_final))

1489
0.9707961309523809
10438
314
10752


In [3]:
def print_stats(df):
    stats = df["constraint_wise_presence"].to_list()
    print(len(stats))
    stat_final =[]
    for stat in stats:
        stat_final.extend(stat)
    print(sum(stat_final)/len(stat_final))
    print(sum(stat_final))
    print(len(stat_final)-sum(stat_final))
    print(len(stat_final))
    

In [51]:
print_stats(df)

1489
0.9707961309523809
10438
314
10752


In [8]:
import pandas as pd
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v9_v2.jsonl", lines=True)
print_stats(df)

1489
0.9738701878370839
10473
281
10754


In [9]:
df["filtered_relevant_constraints"]= [eval(x) for x in df["filtered_relevant_constraints"]]
df.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v9_v2.jsonl", orient="records", lines=True)

In [ ]:
# import pandas as pd
# df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v8.jsonl", lines=True)
# print_stats(df)

1489
0.9619711761971176
10346
409
10755


In [7]:
filter_df_2 = df[df["constraint_wise_presence"].apply(lambda x: sum(x)<len(x))]
filter_df_2.info()
stats = filter_df_2["constraint_wise_presence"].to_list()
print(len(stats))
stat_final =[]  
for stat in stats:
    stat_final.extend(stat)
print(sum(stat_final)/len(stat_final))
print(sum(stat_final))
print(len(stat_final)-sum(stat_final))
print(len(stat_final))
filter_df_2 = filter_df_2.drop(columns=["final_constraints"])
filter_df_2["filtered_relevant_constraints"] = [eval(x) for x in filter_df_2["filtered_relevant_constraints"]]
filter_df_2.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/response_outputs/testing_responses/04_combine_instruct_eval_benchmark_v9.jsonl", orient="records", lines=True)

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 30 to 1449
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               153 non-null    int64  
 1   dataset                          153 non-null    object 
 2   instruction                      153 non-null    object 
 3   code                             153 non-null    object 
 4   test                             129 non-null    object 
 5   relevant_categories              153 non-null    object 
 6   simplified_instruction           153 non-null    object 
 7   extracted_constraints            153 non-null    object 
 8   final_comprehensive_constraints  153 non-null    object 
 9   filtered_relevant_constraints    153 non-null    object 
 10  quality_scores                   153 non-null    object 
 11  relevance_score                  153 non-null    float64
 12  objectivity_score        

In [9]:
filter_df_3 = filter_df_2[filter_df_2["dataset"]=="xlangai/DS-1000"]
filter_df_3.info()
print_stats(filter_df_3)

<class 'pandas.core.frame.DataFrame'>
Index: 109 entries, 375 to 588
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               109 non-null    int64  
 1   dataset                          109 non-null    object 
 2   instruction                      109 non-null    object 
 3   code                             109 non-null    object 
 4   test                             109 non-null    object 
 5   relevant_categories              109 non-null    object 
 6   simplified_instruction           109 non-null    object 
 7   extracted_constraints            109 non-null    object 
 8   final_comprehensive_constraints  109 non-null    object 
 9   filtered_relevant_constraints    109 non-null    object 
 10  quality_scores                   109 non-null    object 
 11  relevance_score                  109 non-null    float64
 12  objectivity_score        

In [32]:
filter_df_3.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/response_outputs/testing_responses/ds_1000_combine_instruct_eval_benchmark_v9_filtered.jsonl", orient="records", lines=True)

In [5]:
filter_df_3 = df[df["dataset"]=="xlangai/DS-1000"]
filter_df_3.info()
print_stats(filter_df_3)

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 375 to 589
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               215 non-null    int64  
 1   dataset                          215 non-null    object 
 2   instruction                      215 non-null    object 
 3   code                             215 non-null    object 
 4   test                             215 non-null    object 
 5   relevant_categories              215 non-null    object 
 6   simplified_instruction           215 non-null    object 
 7   extracted_constraints            215 non-null    object 
 8   final_comprehensive_constraints  215 non-null    object 
 9   filtered_relevant_constraints    215 non-null    object 
 10  quality_scores                   215 non-null    object 
 11  relevance_score                  215 non-null    float64
 12  objectivity_score        

In [11]:
stats = filter_df_3["constraint_wise_presence"].to_list()
print(len(stats))
stat_final =[]
for stat in stats:
    stat_final.extend(stat)
print(sum(stat_final)/len(stat_final))
print(sum(stat_final))
print(len(stat_final)-sum(stat_final))
print(len(stat_final))

    

109
0.6562009419152276
418
219
637


In [ ]:
l1=[1,2]
l2=[3,4]
l3=l1+l2
l3

In [ ]:
import json
import pandas as pd

# Read entire file and split into chunks (one per object)
with open("benchmark_dataset/benchmark_v5.jsonl", "r") as f:
    content = f.read()

blocks = content.strip().split('\n\n')
data = []

# Parse each block into JSON
for idx,block in enumerate(blocks):
    try:
        data.append(json.loads(block))
    except Exception as e:
        print("index: ",idx)
        print("Failed to parse block:\n", block)
        print("Error:", e)

# Create DataFrame
df = pd.DataFrame(data)
df.info()


# Filterig the constraints not in combined instruction 

In [19]:
import pandas as pd 
df = pd.read_json("benchmark_dataset/benchmark_v9_v2.jsonl", lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [25]:
import ast
import traceback

def analyze_false_constraints(
    df,
    constraint_col="filtered_relevant_constraints",
    presence_col="constraint_wise_presence"
):
    # Counters
    extracted_false = 0
    newly_generated_false = 0
    total_false = 0
    track =0 
    for idx, row in df.iterrows():
        try:
            # Read raw values
            constraints_raw = row[constraint_col]
            presence_raw = row[presence_col]

            # Clean markdown formatting if needed
            if isinstance(constraints_raw, str):
                constraints_raw = constraints_raw.replace("```json", "").replace("```", "").strip()
            if isinstance(presence_raw, str):
                presence_raw = presence_raw.replace("```json", "").replace("```", "").strip()

            # Parse
            constraints = ast.literal_eval(constraints_raw) if isinstance(constraints_raw, str) else constraints_raw
            
            # presence = ast.literal_eval(presence_raw) if isinstance(presence_raw, str) else presence_raw
            presence = [p for p in presence_raw if p]

                
            if  len(constraints) != len(presence):
                
                print(f"Row with id {row['id']} has length mismatch: {len(constraints)} constraints but {len(presence)} presence values")
                # row[presence_col].append(True)
                # df.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v9_v2.jsonl", orient="records", lines=True)
                print(f"Length mismatch at row {idx}")
                continue
          

            # Count false cases
            for c, p in zip(constraints, presence):
                if not p:
                    total_false += 1
                    if c["instruction_part"] == "Extracted from instruction":
                        extracted_false += 1
                    elif c["instruction_part"] == "Newly Generated":
                        newly_generated_false += 1

        except Exception as e:
            traceback.print_exc()
            print(f"\nError at row {idx}: {e}")
            print("Raw constraints:", row[constraint_col])
            print("Raw presence:   ", row[presence_col])
            continue

    # Compute percentages
    if total_false > 0:
        perc_extracted = (extracted_false / total_false) * 100
        perc_generated = (newly_generated_false / total_false) * 100
    else:
        perc_extracted = perc_generated = 0

    # Print result
    print(f"\nTotal 'False' constraints: {total_false}")
    print(f"Extracted from instruction: {extracted_false} ({perc_extracted:.2f}%)")
    print(f"Newly Generated: {newly_generated_false} ({perc_generated:.2f}%)")
    print("track: ",track)

    return {
        "total_false": total_false,
        "extracted_false": extracted_false,
        "newly_generated_false": newly_generated_false,
        "perc_extracted": perc_extracted,
        "perc_generated": perc_generated
    }


In [15]:
import re
def extract_aligns_values(text):
    # Use regex to find all occurrences of "Aligns": true or "Aligns": false
    matches = re.findall(r'"Aligns"\s*:\s*(true|false)', text, re.IGNORECASE)
    # Convert string values to boolean
    return [m.lower() == 'true' for m in matches]

In [16]:
text = """ {\n      \"Constraint\": \\\"The function must utilize the 'psutil' library to check for running processes.\\\",\n      \\\"Reason\\\": \\\"The instruction requires using the 'psutil' library to check if the process is running.\\\",\n      \\\"Aligns\\\": true\n    }"""
extract_aligns_values(text)

[]

In [21]:
result = analyze_false_constraints(df)



Total 'False' constraints: 281
Extracted from instruction: 22 (7.83%)
Newly Generated: 257 (91.46%)
track:  0


In [22]:
df = df.drop(columns=["final_constraints"])

In [24]:
import ast

def filter_constraints(row):
    try:
        # Clean and parse
        constraints_raw = row["filtered_relevant_constraints"]
        presence_raw = row["constraint_wise_presence"]

        if isinstance(constraints_raw, str):
            constraints_raw = constraints_raw.replace("```json", "").replace("```", "").strip()
        if isinstance(presence_raw, str):
            presence_raw = presence_raw.replace("```json", "").replace("```", "").strip()

        constraints = ast.literal_eval(constraints_raw) if isinstance(constraints_raw, str) else constraints_raw
        presence = presence_raw

        # Filter where presence is True
        filtered = [c for c, p in zip(constraints, presence) if p]
        return filtered

    except Exception as e:
        print(f"Error in row: {e}")
        return []

# Apply to DataFrame
df["final_constraints"] = df.apply(filter_constraints, axis=1)


In [30]:
result = analyze_false_constraints(df_2,"final_constraints")


Total 'False' constraints: 0
Extracted from instruction: 0 (0.00%)
Newly Generated: 0 (0.00%)
track:  0


In [28]:
df.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v9_v2.jsonl", orient="records", lines=True)

In [ ]:
result = analyze_false_constraints(df,"final_constraints")


In [ ]:
output_path = "benchmark_dataset/benchmark_v7.jsonl"
df.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:
# Find rows where final_constraints is an empty list
empty_final_constraints_df = df[df["final_constraints"].apply(lambda x: isinstance(x, list) and len(x) == 0)]

# Display count and optionally inspect them
print(f"Total rows with empty final_constraints: {len(empty_final_constraints_df)}")

# Optional: Show some rows for inspection
print(empty_final_constraints_df[["id", "instruction", "final_constraints"]].head())


In [ ]:
{
    "id": 412,
    "dataset": "xlangai\/DS-1000",
    "instruction": "Problem:\nI have pandas df with say, 100 rows, 10 columns, (actual data is huge). I also have row_index list which contains, which rows to be considered to take sum. I want to calculate sum on say columns 2,5,6,7 and 8. Can we do it with some function for dataframe object?\nWhat I know is do a for loop, get value of row for each element in row_index and keep doing sum. Do we have some direct function where we can pass row_list, and column_list and axis, for ex df.sumAdvance(row_list,column_list,axis=0) ?\nI have seen DataFrame.sum() but it didn't help I guess.\n  a b c d q \n0 1 2 3 0 5\n1 1 2 3 4 5\n2 1 1 1 6 1\n3 1 0 0 0 0\n\n\nI want sum of 0, 2, 3 rows for each a, b, d columns \na    3.0\nb    3.0\nd    6.0\n\n\nA:\n<code>\nimport pandas as pd\n\n\ndf = pd.DataFrame({'a':[1,1,1,1],'b':[2,2,1,0],'c':[3,3,1,0],'d':[0,4,6,0],'q':[5,5,1,0]})\nrow_list = [0,2,3]\ncolumn_list = ['a','b','d']\n<\/code>\nresult = ... # put solution in this variable\nBEGIN SOLUTION\n<code>",
    "code": "def g(df, row_list, column_list):\n    return df[column_list].iloc[row_list].sum(axis=0)\n\nresult = g(df.copy(), row_list, column_list)",
    "test": "import pandas as pd\nimport numpy as np\nimport copy\nimport tokenize, io\n\n\ndef generate_test_case(test_case_id):\n    def generate_ans(data):\n        data = data\n        df, row_list, column_list = data\n        return df[column_list].iloc[row_list].sum(axis=0)\n\n    def define_test_input(test_case_id):\n        if test_case_id == 1:\n            df = pd.DataFrame(\n                {\n                    \"a\": [1, 1, 1, 1],\n                    \"b\": [2, 2, 1, 0],\n                    \"c\": [3, 3, 1, 0],\n                    \"d\": [0, 4, 6, 0],\n                    \"q\": [5, 5, 1, 0],\n                }\n            )\n            row_list = [0, 2, 3]\n            column_list = [\"a\", \"b\", \"d\"]\n        if test_case_id == 2:\n            df = pd.DataFrame(\n                {\n                    \"a\": [1, 1, 1, 1],\n                    \"b\": [2, 2, 1, 0],\n                    \"c\": [3, 3, 1, 0],\n                    \"d\": [0, 4, 6, 0],\n                    \"q\": [5, 5, 1, 0],\n                }\n            )\n            row_list = [0, 1, 3]\n            column_list = [\"a\", \"c\", \"q\"]\n        return df, row_list, column_list\n\n    test_input = define_test_input(test_case_id)\n    expected_result = generate_ans(copy.deepcopy(test_input))\n    return test_input, expected_result\n\n\ndef exec_test(result, ans):\n    try:\n        pd.testing.assert_series_equal(result, ans, check_dtype=False)\n        return 1\n    except:\n        return 0\n\n\nexec_context = r\"\"\"\nimport pandas as pd\nimport numpy as np\ndf, row_list, column_list = test_input\n[insert]\n\"\"\"\n\n\ndef test_execution(solution: str):\n    code = exec_context.replace(\"[insert]\", solution)\n    for i in range(2):\n        test_input, expected_result = generate_test_case(i + 1)\n        test_env = {\"test_input\": test_input}\n        exec(code, test_env)\n        assert exec_test(test_env[\"result\"], expected_result)\n\n\ndef test_string(solution: str):\n    tokens = []\n    for token in tokenize.tokenize(io.BytesIO(solution.encode(\"utf-8\")).readline):\n        tokens.append(token.string)\n    assert \"while\" not in tokens and \"for\" not in tokens",
    "relevant_categories": "['Code Structure and Modularity', 'Input and Output Handling', 'Data Processing and Transformation', 'Library and API Usage', 'Performance and Optimization', 'Documentation and Readability', 'Mathematical Computation']",
    "simplified_instruction": "Problem: I have pandas df with say, 100 rows, 10 columns, (actual data is huge). I also have row_index list which contains, which rows to be considered to take sum. I want to calculate sum on say columns 2,5,6,7 and 8. Can we do it with some function for dataframe object? What I know is do a for loop, get value of row for each element in row_index and keep doing sum. Do we have some direct function where we can pass row_list, and column_list and axis, for ex df.sumAdvance(row_list,column_list,axis=0) ? I have seen DataFrame.sum() but it didn't help I guess. I want sum of 0, 2, 3 rows for each a, b, d columns a    3.0 b    3.0 d    6.0 A:",
    "extracted_constraints": "[{'type': 'Library and API Usage', 'constraint': 'Use a function for dataframe object to calculate the sum.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Library and API Usage', 'constraint': 'The function should allow passing row_list and column_list.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Library and API Usage', 'constraint': 'The function should specify an axis parameter.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Data Processing and Transformation', 'constraint': 'Calculate the sum for specific columns (2, 5, 6, 7, and 8).', 'instruction_part': 'Extracted from instruction'}, {'type': 'Data Processing and Transformation', 'constraint': 'Calculate the sum for rows specified in row_index.', 'instruction_part': 'Extracted from instruction'}]",
    "final_comprehensive_constraints": "[{'type': 'Library and API Usage', 'constraint': 'Use a function for dataframe object to calculate the sum.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Library and API Usage', 'constraint': 'The function should allow passing row_list and column_list.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Library and API Usage', 'constraint': 'The function should specify an axis parameter.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Data Processing and Transformation', 'constraint': 'Calculate the sum for specific columns (2, 5, 6, 7, and 8).', 'instruction_part': 'Extracted from instruction'}, {'type': 'Data Processing and Transformation', 'constraint': 'Calculate the sum for rows specified in row_index.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Performance and Optimization', 'constraint': 'Ensure the function is optimized for large DataFrames with hundreds of thousands of rows.', 'instruction_part': 'Newly Generated'}, {'type': 'Documentation and Readability', 'constraint': 'Include docstrings in the function to explain parameters and return values.', 'instruction_part': 'Newly Generated'}, {'type': 'Input and Output Handling', 'constraint': 'Validate that row_list and column_list contain valid indices and column names before performing the sum.', 'instruction_part': 'Newly Generated'}, {'type': 'Mathematical Computation', 'constraint': 'Ensure the sum calculation handles missing values appropriately, either by ignoring them or filling them with zeros.', 'instruction_part': 'Newly Generated'}, {'type': 'Code Structure and Modularity', 'constraint': 'Structure the function to allow for easy extension, such as adding more operations beyond summation in the future.', 'instruction_part': 'Newly Generated'}]",
    "filtered_relevant_constraints": "[{'type': 'Library and API Usage', 'constraint': 'Use a function for dataframe object to calculate the sum.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Library and API Usage', 'constraint': 'The function should allow passing row_list and column_list.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Library and API Usage', 'constraint': 'The function should specify an axis parameter.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Data Processing and Transformation', 'constraint': 'Calculate the sum for rows specified in row_index.', 'instruction_part': 'Extracted from instruction'}, {'type': 'Input and Output Handling', 'constraint': 'Validate that row_list and column_list contain valid indices and column names before performing the sum.', 'instruction_part': 'Newly Generated'}, {'type': 'Mathematical Computation', 'constraint': 'Ensure the sum calculation handles missing values appropriately, either by ignoring them or filling them with zeros.', 'instruction_part': 'Newly Generated'}]",
    "quality_scores": "{'constraint_evaluations': [{'constraint_text': 'Use a function for dataframe object to calculate the sum.', 'atomicity_score': 5, 'relevance_score': 5, 'objectivity_score': 5, 'reasoning': 'This constraint is atomic as it specifies a single requirement: to use a function for the DataFrame object. It is highly relevant to the task of calculating a sum using a DataFrame and is objective since it can be clearly evaluated by checking if a function is used.'}, {'constraint_text': 'The function should allow passing row_list and column_list.', 'atomicity_score': 5, 'relevance_score': 5, 'objectivity_score': 5, 'reasoning': \"This constraint is atomic as it focuses on a single requirement regarding the function's parameters. It is relevant because it directly addresses the need to specify which rows and columns to sum. It is also objective, as it can be verified by checking the function's signature.\"}, {'constraint_text': 'The function should specify an axis parameter.', 'atomicity_score': 5, 'relevance_score': 5, 'objectivity_score': 5, 'reasoning': 'This constraint is atomic, focusing solely on the requirement for an axis parameter. It is relevant to the task since the axis is crucial for determining how the sum is calculated. It is objective, as the presence of an axis parameter can be easily checked.'}, {'constraint_text': 'Calculate the sum for rows specified in row_index.', 'atomicity_score': 5, 'relevance_score': 5, 'objectivity_score': 5, 'reasoning': 'This constraint is atomic as it specifies a single requirement regarding the rows to be summed. It is relevant because it directly relates to the task of summing specific rows. It is objective, as it can be verified by checking the implementation of the sum calculation.'}, {'constraint_text': 'Validate that row_list and column_list contain valid indices and column names before performing the sum.', 'atomicity_score': 5, 'relevance_score': 4, 'objectivity_score': 5, 'reasoning': 'This constraint is atomic as it focuses on a single validation requirement. It is relevant, but slightly less so than others, as it pertains to input validation rather than the core summation task. It is objective, as the validation can be clearly defined and checked.'}, {'constraint_text': 'Ensure the sum calculation handles missing values appropriately, either by ignoring them or filling them with zeros.', 'atomicity_score': 5, 'relevance_score': 4, 'objectivity_score': 5, 'reasoning': 'This constraint is atomic as it specifies a single requirement regarding handling missing values. It is relevant, but slightly less so than others, as it addresses a specific case rather than the main functionality. It is objective, as the handling of missing values can be clearly defined and tested.'}], 'avg_atomicity': 5.0, 'avg_relevance': 4.67, 'avg_objectivity': 5.0, 'unified_quality_score': 4.89, 'overall_analysis': 'The overall quality of the constraint set is high, with all constraints scoring well in atomicity and objectivity. Relevance is also strong, though slightly lower for the last two constraints, which focus on validation and handling of missing values rather than the core summation functionality. The constraints are clear, measurable, and directly aligned with the task, making them effective for guiding the implementation.'}",
    "relevance_score": 4.67,
    "objectivity_score": 5.0,
    "atomicity_score": 5.0,
    "unified_quality_score": 4.89,
    "combined_instruction": "Problem:\nI have pandas df with say, 100 rows, 10 columns, (actual data is huge). I also have row_index list which contains, which rows to be considered to take sum. I want to calculate sum on say columns 2,5,6,7 and 8. Can we do it with some function for dataframe object to calculate the sum? The function should allow passing row_list and column_list, and specify an axis parameter. What I know is do a for loop, get value of row for each element in row_index and keep doing sum. Do we have some direct function where we can pass row_list, and column_list and axis, for ex df.sumAdvance(row_list,column_list,axis=0) ? I have seen DataFrame.sum() but it didn't help I guess. \n\nI want sum of 0, 2, 3 rows for each a, b, d columns, ensuring that the sum calculation handles missing values appropriately, either by ignoring them or filling them with zeros. Additionally, validate that row_list and column_list contain valid indices and column names before performing the sum.\n\na    3.0\nb    3.0\nd    6.0\n\nA:\n<code>\nimport pandas as pd\n\n\ndf = pd.DataFrame({'a':[1,1,1,1],'b':[2,2,1,0],'c':[3,3,1,0],'d':[0,4,6,0],'q':[5,5,1,0]})\nrow_list = [0,2,3]\ncolumn_list = ['a','b','d']\n<\/code>\nresult = ... # put solution in this variable\nBEGIN SOLUTION\n<code>\n    \n<\/code>",
    "constraint_wise_presence": [
        false,
        false,
        false,
        false,
        false,
        false
    ],
    "constraint_presence_response": "```json\n{\n  \"Evaluation\": [\n    {\n      \"Constraint\": \"Use a function for dataframe object to calculate the sum.\",\n      \"Reason\": \"The instruction mentions the desire for a direct function to calculate the sum, such as df.sumAdvance, but does not provide an actual function implementation.\",\n      \"Aligns\": false\n    },\n    {\n      \"Constraint\": \"The function should allow passing row_list and column_list.\",\n      \"Reason\": \"The instruction specifies that the function should allow passing row_list and column_list, but does not provide an implementation of such a function.\",\n      \"Aligns\": false\n    },\n    {\n      \"Constraint\": \"The function should specify an axis parameter.\",\n      \"Reason\": \"The instruction mentions the need for an axis parameter in the function, but does not provide an implementation of such a function.\",\n      \"Aligns\": false\n    },\n    {\n      \"Constraint\": \"Calculate the sum for rows specified in row_index.\",\n      \"Reason\": \"The instruction specifies the need to calculate the sum for specific rows using row_index, but does not provide an implementation of this calculation.\",\n      \"Aligns\": false\n    },\n    {\n      \"Constraint\": \"Validate that row_list and column_list contain valid indices and column names before performing the sum.\",\n      \"Reason\": \"The instruction mentions the need to validate row_list and column_list, but does not provide an implementation of this validation.\",\n      \"Aligns\": false\n    },\n    {\n      \"Constraint\": \"Ensure the sum calculation handles missing values appropriately, either by ignoring them or filling them with zeros.\",\n      \"Reason\": \"The instruction specifies handling missing values appropriately, but does not provide an implementation of this handling.\",\n      \"Aligns\": false\n    }\n  ]\n}\n```"
}

In [ ]:
df = df[df["final_constraints"].apply(lambda x: not (isinstance(x, list) and len(x) == 0))]
output_path = "benchmark_dataset/benchmark_v7.jsonl"
df.to_json(output_path, orient="records", lines=True, force_ascii=False)

In [ ]:
df.info()

In [ ]:
import pandas as pd
df = pd.read_json("benchmark_dataset/benchmark_v7.jsonl",lines=True)
df.info()

In [ ]:
stats = df["constraint_wise_presence"].to_list()
print(len(stats))
stat_final =[]
for stat in stats:
    stat_final.extend(stat)
print(sum(stat_final)/len(stat_final))
print(len(stat_final))

In [ ]:
constraints = df["final_constraints"].to_list()
print(constraints[0])
print(len(constraints))
total = []
for i in constraints:
    total.extend(i)
print(len(total))
print(total[0])

In [ ]:
for idx, row in df.iterrows():
    num_true = sum(row["constraint_wise_presence"])
    num_constraints = len(row["final_constraints"])
    if num_true != num_constraints:
        print(f"Mismatch at row {idx}: {num_true} True vs {num_constraints} constraints")


In [ ]:
# Flatten presence lists and count True values
stats = df["constraint_wise_presence"].to_list()
stat_final = []
for stat in stats:
    stat_final.extend(stat)
total_true = sum(stat_final)

# Flatten final_constraints and count entries
constraints = df["final_constraints"].to_list()
total_constraints = []
for c_list in constraints:
    total_constraints.extend(c_list)
total_len = len(total_constraints)

# Print results
print(f"Total number of `True` values in constraint_wise_presence: {total_true}")
print(f"Total number of constraints in final_constraints: {total_len}")
print(f"Do they match? {'✅ Yes' if total_true == total_len else '❌ No'}")

# Optional: check mismatched rows
mismatches = []
for idx, row in df.iterrows():
    num_true = sum(row["constraint_wise_presence"])
    num_constraints = len(row["final_constraints"])
    if num_true != num_constraints:
        mismatches.append((idx, num_true, num_constraints))

print(f"Total mismatched rows: {len(mismatches)}")
if mismatches:
    print("Sample mismatches (index, true_count, constraint_count):")
    print(mismatches[:5])


In [ ]:
import re

def extract_alignment_scores_regex(response):
    results = []
    pattern = re.compile(r'"Aligns"\s*:\s*(true|false)', re.IGNORECASE)


    try:
        matches = pattern.findall(response)
        aligns_row = [1 if m.lower() == "true" else 0 for m in matches]
        results.append(aligns_row)
    except Exception as e:
        print(f"[!] Error parsing response: {e}")
        print(response)
      

    return results

In [ ]:
import re

def extract_alignment_scores_from_evaluation(response_list):
    results = []

    for response in response_list:
        try:
            # Extract the Evaluation block only
            eval_block_match = re.search(r'"Evaluation"\s*:\s*\[(.*?)\](?=,|\s*["}])', response, re.DOTALL)
            if not eval_block_match:
        
                results.append([])
                continue

            eval_block = eval_block_match.group(1)

            # Now extract Aligns values inside this block
            aligns = re.findall(r'"Aligns"\s*:\s*(true|false)', eval_block, flags=re.IGNORECASE)
            aligns_row = [1 if val.lower() == "true" else 0 for val in aligns]
            results.append(aligns_row)

        except Exception as e:
      
            results.append([])

    return results


In [ ]:
response =[ """{'Evaluation': [{'Constraint': 'The `PluginManager` should be able to register multiple versions of plugins.', 'Reason': 'The `PluginManager` class has a `register_plugin` method that adds plugins to a dictionary keyed by their version, allowing multiple plugin versions to be registered.', 'Aligns': True}, {'Constraint': 'Each plugin version should have a unique identifier.', 'Reason': "Each plugin instance has a `version` attribute used as a unique key in the `PluginManager`'s `plugins` dictionary, ensuring uniqueness.", 'Aligns': True}, {'Constraint': 'The `PluginManager` should provide a method to get a plugin by its version identifier.', 'Reason': 'The `get_plugin` method in `PluginManager` retrieves a plugin by its version identifier.', 'Aligns': True}, {'Constraint': 'The `PluginManager` should provide a method to list all registered plugins along with their capabilities.', 'Reason': 'The `list_plugins` method returns a dictionary mapping plugin versions to their capabilities.', 'Aligns': True}, {'Constraint': 'Implement error handling for cases where a plugin version is not found.', 'Reason': 'The `get_plugin` method raises a `ValueError` with a descriptive message if the requested plugin version is not found.', 'Aligns': True}, {'Constraint': 'Use the provided `Plugin_v1` class as a base class for creating a new plugin version `Plugin_v2`.', 'Reason': '`Plugin_v2` inherits from `Plugin_v1` as shown in the class definition.', 'Aligns': True}, {'Constraint': 'The `Plugin_v2` should override the `capabilities` method to return its new capabilities.', 'Reason': '`Plugin_v2` overrides the `capabilities` method to return its capabilities set, which includes the added import and export capabilities.', 'Aligns': True}, {'Constraint': 'The `PluginManager` should raise a descriptive exception if an attempt is made to register a plugin with an existing version identifier.', 'Reason': 'The `register_plugin` method raises a `ValueError` with a descriptive message if a plugin with the same version is already registered.', 'Aligns': True}], 'FinalDecision': {'Score': '8/8', 'Reason': 'Yes, all constraints were satisfied as the response implements all required features and error handling as specified.', 'Aligns': True}}
"""]
extract_alignment_scores_from_evaluation(response)[0]

In [ ]:
response =[ """{'Evaluation': [{'Constraint': 'The `PluginManager` should be able to register multiple versions of plugins.', 'Reason': 'The `PluginManager` class has a `register_plugin` method that adds plugins to a dictionary keyed by their version, allowing multiple plugin versions to be registered.', 'Aligns': True}, {'Constraint': 'Each plugin version should have a unique identifier.', 'Reason': "Each plugin instance has a `version` attribute used as a unique key in the `PluginManager`'s `plugins` dictionary, ensuring uniqueness.", 'Aligns': True}, {'Constraint': 'The `PluginManager` should provide a method to get a plugin by its version identifier.', 'Reason': 'The `get_plugin` method in `PluginManager` retrieves a plugin by its version identifier.', 'Aligns': True}, {'Constraint': 'The `PluginManager` should provide a method to list all registered plugins along with their capabilities.', 'Reason': 'The `list_plugins` method returns a dictionary mapping plugin versions to their capabilities.', 'Aligns': True}, {'Constraint': 'Implement error handling for cases where a plugin version is not found.', 'Reason': 'The `get_plugin` method raises a `ValueError` with a descriptive message if the requested plugin version is not found.', 'Aligns': True}, {'Constraint': 'Use the provided `Plugin_v1` class as a base class for creating a new plugin version `Plugin_v2`.', 'Reason': '`Plugin_v2` inherits from `Plugin_v1` as shown in the class definition.', 'Aligns': True}, {'Constraint': 'The `Plugin_v2` should override the `capabilities` method to return its new capabilities.', 'Reason': '`Plugin_v2` overrides the `capabilities` method to return its capabilities set, which includes the added import and export capabilities.', 'Aligns': True}, {'Constraint': 'The `PluginManager` should raise a descriptive exception if an attempt is made to register a plugin with an existing version identifier.', 'Reason': 'The `register_plugin` method raises a `ValueError` with a descriptive message if a plugin with the same version is already registered.', 'Aligns': True}], 'FinalDecision': {'Score': '8/8', 'Reason': 'Yes, all constraints were satisfied as the response implements all required features and error handling as specified.', 'Aligns': True}}
"""]
print(extract_alignment_scores_from_evaluation(response)[0])

In [ ]:
import pandas as pd
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/metrics_outputs/granite-3.1-2b-instruct_results_correctness_metrics.jsonl",lines=True)
df.info()

In [ ]:
df.columns

In [ ]:
n = 2
sampled_df = df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(n=min(n, len(x))))


In [ ]:
sampled_df.info()

In [ ]:
sampled_df.to_json("response_outputs/testing_responses/granite-3.1-2b-instruct_results_correctness_metrics.jsonl",orient='records',lines=True)

In [ ]:
import json

def read_jsonl_file(file_path):
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

def print_row_columns(data, row_num):
    if row_num < 0 or row_num >= len(data):
        print(f"Invalid row number. Please choose between 0 and {len(data) - 1}.")
        return
    row = data[row_num]
    print(f"Row {row_num}:")
    for key, value in row.items():
        print(f"{key}: {value}")

if __name__ == "__main__":
    file_path = "/dccstor/shanmukh/sravani_internship/benchmark_experiments/response_outputs/testing_responses/granite-3.1-2b-instruct_results_correctness_metrics.jsonl"  # Replace with your actual .jsonl file path
    row_number = int(input("Enter the row number: "))

    data = read_jsonl_file(file_path)
    print_row_columns(data, row_number)


In [ ]:
 print_row_columns(data, 2)

# New Prompt for LLM as Judge

In [ ]:
import json
from typing import List
from utils.compute_clients import LLMClient
from utils.compute_clients import create_clients


def evaluate_response_strict_alignment(instruction: str, constraints: List[str], response: str, client: LLMClient) -> dict:
    """Evaluate whether a response strictly satisfies each constraint in the list using an LLM verifier."""
    
    # Build the strict verifier system prompt
    system_prompt = """You are a verifier. Your task is to evaluate whether a given response satisfies a set of constraints for a specific instruction.

You will be provided:
- An instruction
- A list of constraints
- A response to the instruction

Your task:
- Analyze the response against each constraint independently.
- For each constraint, determine whether it is strictly and completely satisfied.
- If any part of a constraint is only partially fulfilled or ambiguous, mark it as not satisfied.
- Provide a brief but clear explanation of your judgment for each constraint.
- Do not make assumptions beyond the constraint. Only base your judgment on what is explicitly implemented or stated in the response.

Output your evaluation in a valid JSON format like below:
{
  "Evaluation": [
    {
      "Constraint": "<constraint text>",
      "Reason": "explanation",
      "Aligns": [true|false]
    },
    ...
  ]
}

Do not include any text outside this JSON object."""

    # Create the user prompt
    user_prompt = f"""[Instruction]:
{instruction}

[Constraints]:
{json.dumps(constraints, indent=2)}

[Response]:
```python
{response}
```"""

    # Send to the model and return the parsed JSON output
    model_output = client.get_model_response(
        user_prompt=user_prompt,
        system_prompt=system_prompt,
        temperature=0,
        max_new_tokens=2048
    )

    try:
        return json.loads(model_output)
    except json.JSONDecodeError:
        return {"error": "Model output was not valid JSON", "raw_output": model_output}


# New Prompt for code correctness

In [106]:
import json
from typing import List
from utils.compute_clients import LLMClient
from utils.compute_clients import create_clients


def evaluate_response_code_correctness(instruction: str, constraints: List[str], response: str, client: LLMClient) -> dict:
    prompt = f"""You are an expert Python developer and uncompromising code reviewer.  
Your task is to evaluate whether the provided Python code strictly and robustly follows the instruction, based only on three dimensions:

1. Syntax Correctness:  
   - The code must parse and compile without any syntax errors.

2. Semantic Correctness:  
   - The code must run and produce the intended effect exactly as described.  
   - Any hidden bugs or logical mistakes count as failures.

3. Constraint Correctness:  
   - The code must satisfy *every* constraint explicitly stated in the instruction, including any edge-case, privacy, or consistency requirements.  
   - If the instruction implies or adds new requirements (e.g. key ordering, redaction of sensitive data), those must be met exactly.

Important Review Principles:
- **Be conservative.** Only label “Completely Correct” if you are *absolutely certain* that all dimensions are fully satisfied—no exceptions.  
- **Downgrade on any violation.**  
  - If syntax or runtime errors exist, mark as **Wrong**.  
  - If the code runs but omits or mis-implements any single required constraint, mark as **Partially Correct** at best.  
- **No leniency for extra features.** Any extra functionality that interferes with, overrides, or exposes data beyond the instruction is a constraint violation.

Evaluation Labels:
- **Completely Correct**: Perfect in syntax, semantics, and constraint adherence—no deviations.  
- **Partially Correct**: Runs without errors but misses or mis-implements one or more explicit requirements.  
- **Wrong**: Contains syntax/runtime errors or fails to implement the core instruction at all.

**If you have any doubt about full compliance, choose the lower category.**

Return *only* this dictionary—no further explanation:
{{  
  "reason": "<Concise justification for your label>",  
  "correctness": "Completely Correct" | "Partially Correct" | "Wrong"  
}}
Input:
Instruction:  
{instruction}

Generated Code:  
```python
{response}
```
    """



    # Send to the model and return the parsed JSON output
    model_output = client.get_model_response(
        user_prompt=prompt,
        system_prompt=prompt,
        temperature=0,
        max_new_tokens=2048
    )

    try:
        return json.loads(model_output)
    except json.JSONDecodeError:
        return {"error": "Model output was not valid JSON", "raw_output": model_output}


In [36]:
instruction =before_file["combined_instruction"][3]
constraints = before_file["final_constraints"][3]
response = before_file["response"][3]

client = create_clients(mode="GPT-azure")
result = evaluate_response_code_correctness(instruction, constraints, response, client)
print(result)


{'reason': "The code is syntactically correct and runs without errors. It correctly returns 'Not Prime' for non-integers (including decimals), negative numbers, and numbers <= 1. It returns 'Prime' or 'Not Prime' exactly as required. It uses O(sqrt(n)) time complexity, checks only odd divisors after 2, and uses constant space. All constraints, including handling of decimal inputs and skipping even numbers > 2, are met exactly.", 'correctness': 'Completely Correct'}


In [ ]:
def extract_aligns_values(text):

    # Use regex to find all occurrences of "Aligns": true or "Aligns": false
    matches = re.findall(r'"[Aligns]"\s*:\s*(true|false)', text, re.IGNORECASE)
    # Convert string values to boolean
    return [m.lower() == 'true' for m in matches]

In [67]:
def extract_correctness_values(text):

    # Use regex to find all occurrences of "correctness": "Completely Correct", "Partially Correct", or "Wrong"
    matches = re.findall(r'"correctness"\s*:\s*("(?:Completely Correct|Partially Correct|Wrong)")', text, re.IGNORECASE)
    # Extract the matched strings and return them
    return matches[0].strip('"') 

In [68]:
text = """```json                                                                                              
{                                                                                                                                                                                          
  "reason": "The code contains multiple critical issues: The 'routes.py' file attempts to access 'socketio.get_store().get_totals()', but 'socketio' does not have a 'get_store()' method, 
causing runtime errors. The 'handle_connect' event handler is defined in 'routes.py' but never imported or registered, so clients will not receive the initial vote totals on connection, v
iolating the requirement that the HTML page displays current vote totals and updates in real-time. In 'index.html', the JavaScript incorrectly updates all three <p> elements with the same
 combined text, causing display errors. The vote form's submit button lacks type="submit", and the form submission is not properly handled to prevent page reload. The tests in 'tests/test
_routes.py' are nonsensical and do not test the actual application logic. These issues mean the code will not run correctly or produce the intended effect exactly as described, failing se
mantic and constraint correctness. Syntax is mostly correct, but runtime errors and missing features downgrade correctness to Partially Correct.",                                         
  "correctness": "Completely Correct"                                                                                                                                                       
}                                                                                                                                                                                          
```  """
print(extract_correctness_values(text))

Completely Correct


In [70]:
after_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [71]:
after_df["correctness_level"] = after_df["code_correctness_response"].apply(extract_correctness_values)


In [63]:
after_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [ ]:
import os
import dotenv
dotenv.load_dotenv()

In [ ]:
sampled_df.info()

In [ ]:
df.columns

In [ ]:
sampled_df = sampled_df.reset_index()

In [ ]:
response = sampled_df["response"][0]
response

In [ ]:
instruction =df["combined_instruction"][3]
constraints = df["final_constraints"][3]
response = df["response"][3]

client = create_clients(mode="GPT-azure")
result = evaluate_response_strict_alignment(instruction, constraints, response, client)
print(result)


## evaluate_row_constraints_strict

In [104]:
import json
from typing import Tuple
from utils.compute_clients import create_clients, LLMClient

def evaluate_row_constraints_strict(n: int, df) -> Tuple[dict, list, str]:
    """Evaluate constraint adherence for row `n` in the dataframe using strict alignment check."""

    # Extract instruction, constraints, and response
    print("ID:",n)
    print("dataset:",df["dataset"][n])
    og_instruction = df["instruction"][n]
    print("Og_instruction:\n",og_instruction)
    print("------------------------------------------------------\n")

    instruction = df["combined_instruction"][n]
    print("combined_instruction:\n", instruction)
    print("------------------------------------------------------\n")
    constraints = df["final_constraints"][n]
    for con in constraints:
        print("Constraint:\n", con)
    print("------------------------------------------------------\n")

    response = df["response"][n]
    print("Response: \n",response)

    # print("Previous List :\n", df["Constraint_adherence"][n])

    # # Create client
    # client = create_clients(mode="GPT-azure")

    # # Call the strict evaluation function
    # result = evaluate_response_strict_alignment(instruction, constraints, response, client)

    # # Extract true/false list
    # if "Evaluation" in result:
    #     alignments = [entry.get("Aligns", "nan") for entry in result["Evaluation"]]
    # else:
    #     alignments = []

    # print("Alignment Booleans:", alignments)
    # # print("Response Code:\n", code_response)
    # print("Full Evaluation JSON:\n", )
    # for res in result.get("Evaluation"):
    #     print("reasoning:\n",res)

    print("Correctness: \n",df["correctness_level"][n])

    print("Correctness Reason: \n",df["code_correctness_response"][n])
    # return result, alignments


In [107]:
after_df_wrong = after_df[after_df["correctness_level"] == "Wrong"]
after_df_wrong_ds = after_df_wrong[after_df_wrong["dataset"] == "xlangai/DS-1000"]
after_df_wrong_ds.info()
big_code_df = after_df_wrong[after_df_wrong["dataset"] == "bigcode/bigcodebench"]
big_code_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 377 to 589
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               61 non-null     int64  
 1   dataset                          61 non-null     object 
 2   instruction                      61 non-null     object 
 3   code                             61 non-null     object 
 4   test                             61 non-null     object 
 5   relevant_categories              61 non-null     object 
 6   simplified_instruction           61 non-null     object 
 7   extracted_constraints            61 non-null     object 
 8   final_comprehensive_constraints  61 non-null     object 
 9   filtered_relevant_constraints    61 non-null     object 
 10  quality_scores                   61 non-null     object 
 11  relevance_score                  61 non-null     float64
 12  objectivity_score         

In [109]:
big_code_df = big_code_df.reset_index()
evaluate_row_constraints_strict(2, big_code_df)

ID: 2
dataset: bigcode/bigcodebench
Og_instruction:
 Find the latest log file in a specified directory that matches a given regex pattern. This function searches through all files in the specified directory, filters them based on the provided regex pattern, and returns the path to the most recent log file based on modification time. If no files match the pattern or the directory is empty, the function returns None.
The function should output with:
    str or None: The path to the most recent log file that matches the pattern, or None if no matching files are found.
You should write self-contained code starting with:
```
import os
import re
def task_func(pattern, log_dir='/var/log/'):
```
------------------------------------------------------

combined_instruction:
 Find the latest log file in a specified directory that matches a given regex pattern. This function searches through all files in the specified directory, filters them based on the provided regex pattern, and returns the pat

In [99]:
evaluate_row_constraints_strict(70, after_df_wrong)

dataset: ajibawa-2023/Python-Code-23k-ShareGPT
Og_instruction:
 Please write a code to delete all items from a list with a value of 3 and calculate the sum of the remaining items. Additionally, you should handle cases where the list is empty or contains only items with a value of 3. The time complexity of your solution should be O(n), where n is the length of the list. You are not allowed to use any built-in functions or methods that directly solve this problem (e.g. using filter(), list comprehension, etc.).

mylist = [2, 3, 3, 4, 5, 6, 4, 7]

#Expected Output
The sum of the remaining items after deleting all items with a value of 3 is 31.
------------------------------------------------------

combined_instruction:
 Please write a code to delete all items from a list with a value of 3 and calculate the sum of the remaining items. The function must correctly delete all occurrences of the value 3 from the list. Additionally, you should handle cases where the list is empty or contains o

In [105]:
evaluate_row_constraints_strict(37, after_df_wrong_ds)

ID: 37
dataset: xlangai/DS-1000
Og_instruction:
 Problem:
Let's say I have 5 columns.
pd.DataFrame({
'Column1': [1, 2, 3, 4, 5, 6, 7, 8, 9],
'Column2': [4, 3, 6, 8, 3, 4, 1, 4, 3],
'Column3': [7, 3, 3, 1, 2, 2, 3, 2, 7],
'Column4': [9, 8, 7, 6, 5, 4, 3, 2, 1],
'Column5': [1, 1, 1, 1, 1, 1, 1, 1, 1]})


Is there a function to know the type of relationship each par of columns has? (one-to-one, one-to-many, many-to-one, many-to-many)
An list output like:
['Column1 Column2 one-to-many',
 'Column1 Column3 one-to-many',
 'Column1 Column4 one-to-one',
 'Column1 Column5 one-to-many',
 'Column2 Column1 many-to-one',
 'Column2 Column3 many-to-many',
 'Column2 Column4 many-to-one',
 'Column2 Column5 many-to-many',
 'Column3 Column1 many-to-one',
 'Column3 Column2 many-to-many',
 'Column3 Column4 many-to-one',
 'Column3 Column5 many-to-many',
 'Column4 Column1 one-to-one',
 'Column4 Column2 one-to-many',
 'Column4 Column3 one-to-many',
 'Column4 Column5 one-to-many',
 'Column5 Column1 many-to-one'

In [42]:
before_file = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/LLMjudge_outputs/code_correctness/granite-3.1-8b-instruct_results_correctness.jsonl", lines=True)
before_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [43]:
after_df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/LLMjudge_outputs_test/code_correctness/llama-3.1-8b-instruct_with_correctness.jsonl", lines=True)
after_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [46]:
def compute_correctness(df,colname="correctness_level"):
    completely_correct = df[colname].value_counts().get("Completely Correct", 0)
    partially_correct = df[colname].value_counts().get("Partially Correct", 0)
    wrong = df[colname].value_counts().get("Wrong", 0)
    # Percetages of each correctness level
    total = len(df)
    completely_correct_percentage = (completely_correct / total) * 100 if total > 0 else 0
    partially_correct_percentage = (partially_correct / total) * 100 if total > 0 else 0
    wrong_percentage = (wrong / total) * 100 if total > 0 else 0
    print(f"Completely Correct: {completely_correct} ({completely_correct_percentage:.2f}%)")
    print(f"Partially Correct: {partially_correct} ({partially_correct_percentage:.2f}%)")
    print(f"Wrong: {wrong} ({wrong_percentage:.2f}%)")
    # Per uique value in the dataset column
    unique_values = df["dataset"].unique()
    print(f"Unique datasets: {len(unique_values)}")
    print("Datasets:", unique_values)
    # Print all 3 correctness levels and their percentages for each uique value in the dataset column
    df_2 = df.groupby("dataset")[colname].value_counts(normalize=True).unstack(fill_value=0) * 100
    print("\nCorrectness levels by dataset:")
    # print(df_2)
    # Percentages of each correctness level for each unique value in the dataset column
    for dataset in unique_values:
        dataset_df = df[df["dataset"] == dataset]
        completely_correct = dataset_df[colname].value_counts().get("Completely Correct", 0)
        partially_correct = dataset_df[colname].value_counts().get("Partially Correct", 0)
        wrong = dataset_df[colname].value_counts().get("Wrong", 0)
        total = len(dataset_df)
        completely_correct_percentage = (completely_correct / total) * 100 if total > 0 else 0
        partially_correct_percentage = (partially_correct / total) * 100 if total > 0 else 0
        wrong_percentage = (wrong / total) * 100 if total > 0 else 0
        print(f"\nDataset: {dataset}")
        print(f"Completely Correct: {completely_correct} ({completely_correct_percentage:.2f}%)")
        print(f"Partially Correct: {partially_correct} ({partially_correct_percentage:.2f}%)")
        print(f"Wrong: {wrong} ({wrong_percentage:.2f}%)")
    return [completely_correct, partially_correct, wrong, completely_correct_percentage, partially_correct_percentage, wrong_percentage]


In [ ]:
compute_correctness(before_file, colname="correctness_level")


Completely Correct: 942 (63.26%)
Partially Correct: 128 (8.60%)
Wrong: 419 (28.14%)
Unique datasets: 4
Datasets: ['ajibawa-2023/Python-Code-23k-ShareGPT' 'xlangai/DS-1000'
 'bigcode/bigcodebench' 'Multilingual-Multimodal-NLP/McEval-Instruct']

Correctness levels by dataset:

Dataset: ajibawa-2023/Python-Code-23k-ShareGPT
Completely Correct: 222 (59.20%)
Partially Correct: 39 (10.40%)
Wrong: 114 (30.40%)

Dataset: xlangai/DS-1000
Completely Correct: 127 (59.07%)
Partially Correct: 8 (3.72%)
Wrong: 80 (37.21%)

Dataset: bigcode/bigcodebench
Completely Correct: 276 (73.60%)
Partially Correct: 27 (7.20%)
Wrong: 72 (19.20%)

Dataset: Multilingual-Multimodal-NLP/McEval-Instruct
Completely Correct: 317 (60.50%)
Partially Correct: 54 (10.31%)
Wrong: 153 (29.20%)


[np.int64(317),
 np.int64(54),
 np.int64(153),
 np.float64(60.49618320610687),
 np.float64(10.305343511450381),
 np.float64(29.198473282442748)]

In [72]:
compute_correctness(after_df, colname="correctness_level")

Completely Correct: 475 (31.90%)
Partially Correct: 510 (34.25%)
Wrong: 504 (33.85%)
Unique datasets: 4
Datasets: ['ajibawa-2023/Python-Code-23k-ShareGPT' 'xlangai/DS-1000'
 'bigcode/bigcodebench' 'Multilingual-Multimodal-NLP/McEval-Instruct']

Correctness levels by dataset:

Dataset: ajibawa-2023/Python-Code-23k-ShareGPT
Completely Correct: 119 (31.73%)
Partially Correct: 123 (32.80%)
Wrong: 133 (35.47%)

Dataset: xlangai/DS-1000
Completely Correct: 76 (35.35%)
Partially Correct: 78 (36.28%)
Wrong: 61 (28.37%)

Dataset: bigcode/bigcodebench
Completely Correct: 143 (38.13%)
Partially Correct: 109 (29.07%)
Wrong: 123 (32.80%)

Dataset: Multilingual-Multimodal-NLP/McEval-Instruct
Completely Correct: 137 (26.15%)
Partially Correct: 200 (38.17%)
Wrong: 187 (35.69%)


[np.int64(137),
 np.int64(200),
 np.int64(187),
 np.float64(26.14503816793893),
 np.float64(38.16793893129771),
 np.float64(35.68702290076336)]

In [75]:
after_df.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/LLMjudge_outputs_test/code_correctness/llama-3.1-8b-instruct_with_correctness.jsonl", orient='records', lines=True)

In [ ]:
df = sampled_df.copy()

In [ ]:
result, alignments = evaluate_row_constraints_strict(5, df)


In [ ]:
result, alignments = evaluate_row_constraints_strict(4, df)

In [ ]:
result, alignments = evaluate_row_constraints_strict(5, df)

In [ ]:
result, alignments = evaluate_row_constraints_strict(9, df)

In [ ]:
result, alignments = evaluate_row_constraints_strict(7, df)

In [ ]:
area = 78.65
print("{:.2f}".format(area))

In [ ]:
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True

def find_prime_sum_array(N, M, X):
    prime_sum_array = []
    current_number = M
    sum_so_far = 0

    while len(prime_sum_array) < N and current_number <= X:
        if is_prime(sum_so_far + current_number):
            prime_sum_array.append(current_number)
            sum_so_far += current_number
        current_number += M

    if not is_prime(sum_so_far):
        return []

    return prime_sum_array

# Example usage
N = 5
M = 3
X = 20

result = find_prime_sum_array(N, M, X)
print(result)

In [ ]:
df.columns

In [ ]:
import ast
from collections import Counter

# Assuming df is loaded and final_constraints parsed:
# If not parsed yet, do:
df['final_constraints'] = df['final_constraints'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Gather all instruction_part values
parts = []
for entry in df['final_constraints']:
    for rec in entry:
        parts.append(rec.get('instruction_part'))

# Count frequencies
part_counts = Counter(parts)

print("Instruction Part Counts:")
for part, count in part_counts.items():
    print(f"- {part}: {count}")


In [ ]:
import ast

# 1. Load & parse
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v7.jsonl", lines=True)  # or pd.read_csv(...)
df['final_constraints'] = df['final_constraints'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# 2. Normalize the instruction_part labels in the DataFrame itself
def normalize_parts(recs):
    for rec in recs:
        part = rec.get("instruction_part", "")
        if part.startswith("Original source"):
            rec["instruction_part"] = "Extracted from instruction"
        elif part == "Combined/Refined":
            rec["instruction_part"] = "Newly Generated"
    return recs

df['final_constraints'] = df['final_constraints'].apply(normalize_parts)

# 3. (Optional) Write the cleaned DataFrame back out:
df.to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/benchmark_dataset/benchmark_v7.jsonl", orient="records", lines=True, force_ascii=False)


In [ ]:
import os
import ast
import pandas as pd


def normalize_parts(constraints_list):
    """
    Normalize any stray instruction_part labels in a list of constraint dicts:
      - "Original source: ..." → "Extracted from instruction"
      - "Combined/Refined"    → "Newly Generated"
    """
    for rec in constraints_list:
        part = rec.get("instruction_part", "")
        if part.startswith("Original source"):
            rec["instruction_part"] = "Extracted from instruction"
        elif part == "Combined/Refined":
            rec["instruction_part"] = "Newly Generated"
    return constraints_list


def process_folder(folder_path):
    """
    Traverse the folder, normalize 'final_constraints' in each .jsonl/.csv file,
    and overwrite the files in place.
    """
    for fname in os.listdir(folder_path):
        path = os.path.join(folder_path, fname)
        if not os.path.isfile(path):
            continue
        # load
        if fname.endswith('.jsonl'):
            df = pd.read_json(path, lines=True)
        elif fname.endswith('.csv'):
            df = pd.read_csv(path)
        else:
            continue

        # parse and normalize
        df['final_constraints'] = df['final_constraints'].apply(
            lambda x: normalize_parts(ast.literal_eval(x)) if isinstance(x, str) else normalize_parts(x)
        )

        # save back
        if fname.endswith('.jsonl'):
            df.to_json(path, orient='records', lines=True, force_ascii=False)
        else:
            df.to_csv(path, index=False)

        print(f"Normalized and overwritten: {path}")


if __name__ == '__main__':
    folder = "/dccstor/shanmukh/sravani_internship/benchmark_experiments/outputs/experiments"
    process_folder(folder)


In [ ]:
import pandas as pd
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/metrics_outputs/test/granite-3.1-2b-instruct_results_correctness_metrics_extended.jsonl",lines=True)
df.info()

In [ ]:
df["dataset"].unique()

In [ ]:
bigcode_bench = df[df["dataset"]=="bigcode/bigcodebench"]
ds_1000 = df[df["dataset"]=='xlangai/DS-1000']
share_gpt = df[df["dataset"]=='ajibawa-2023/Python-Code-23k-ShareGPT']
mceval_instruct = df[df["dataset"]=='Multilingual-Multimodal-NLP/McEval-Instruct']


In [ ]:
filter_df.info()

## Compute SSR by intsruction part

In [ ]:
from collections import defaultdict
def compute_ssr_by_instruction_part(df, constraints_col, adherence_col):
    """
    Return a dict mapping each instruction_part
    ("Extracted from instruction" vs "Newly Generated")
    to its overall SSR.
    """
    part_stats = defaultdict(lambda: {"sum": 0, "count": 0})

    for _, row in df.iterrows():
        constraints = (
            ast.literal_eval(row[constraints_col])
            if isinstance(row[constraints_col], str)
            else row[constraints_col]
        )
        flags = row[adherence_col]
        for rec, flag in zip(constraints, flags):
            part = rec["instruction_part"]
            part_stats[part]["sum"] += flag
            part_stats[part]["count"] += 1

    return {part: v["sum"] / v["count"] for part, v in part_stats.items()},{part:v["count"]for part, v in part_stats.items()}

In [ ]:
constraints_col = "final_constraints"
adherence_col = "Constraint_adherence"
result,counts = compute_ssr_by_instruction_part(granite_2b_dfs["ds_1000"],constraints_col,adherence_col)

In [ ]:
result

In [ ]:
counts

In [ ]:
import ast
from collections import defaultdict
def count_correctness(df,correctness_col="correctness_level"):
    count = defaultdict(lambda: {"count": 0})
    for _,row in df.iterrows():
        correctness = row[correctness_col]
        value = f"{correctness.replace(' ', '_')}"
        # print(value)
        count[value]["count"]+=1
        
    return {part: v["count"] for part, v in count.items()},{part:v["count"]/len(df) for part, v in count.items()}
    return count
        

# Filtering datasets

In [1]:
def dataset_wise_filter(df):
    dfs = {}
    dfs["bigcode_bench"] = df[df["dataset"]=="bigcode/bigcodebench"]
    dfs["ds_1000"] = df[df["dataset"]=='xlangai/DS-1000']
    dfs["share_gpt"] = df[df["dataset"]=='ajibawa-2023/Python-Code-23k-ShareGPT']
    dfs["mceval_instruct"] = df[df["dataset"]=='Multilingual-Multimodal-NLP/McEval-Instruct']
    return dfs

In [2]:
import pandas as pd
granite_2b = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/metrics_outputs/test/granite-3.1-2b-instruct_results_correctness_metrics_extended.jsonl",lines=True)
granite_2b.info()
granite_2b_dfs = dataset_wise_filter(granite_2b)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [ ]:
for key, value in granite_2b_dfs.items():
    print(key)
    count = count_correctness(value)
    print(count)

In [ ]:
granite_2b_dfs["bigcode_bench"].to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/response_outputs/testing_responses/bigcode_granite-3.1-2b-instruct_results_correctness_metrics.jsonl",orient="records",lines=True)

In [ ]:
granite_2b_dfs["ds_1000"].to_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/response_outputs/testing_responses/ds_1000_granite-3.1-2b-instruct_results_correctness_metrics.jsonl",orient="records",lines=True)

In [ ]:
import pandas as pd
granite_2b = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/LLMjudge_outputs/code_correctness/Mistral-Small-3.1-24B-Instruct-2503_results_correctness.jsonl",lines=True)
granite_2b.info()
granite_2b_dfs = dataset_wise_filter(granite_2b)

In [ ]:
for key, value in granite_2b_dfs.items():
    print(key)
    value = value.reset_index()
    count = count_correctness(value)
    print(count)

In [ ]:
import pandas as pd
granite_2b = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/LLMjudge_outputs/code_correctness/granite-3.1-8b-instruct_results_correctness.jsonl",lines=True)
granite_2b.info()
granite_2b_dfs = dataset_wise_filter(granite_2b)

In [ ]:
for key, value in granite_2b_dfs.items():
    print(key)
    count = count_correctness(value)
    print(count)

In [ ]:
print("ds_1000")
count = count_correctness(ds_1000)
print(count)

In [ ]:
print("share_gpt")
count = count_correctness(share_gpt)
print(count)

In [ ]:
print("mceval_instruct")
count = count_correctness(mceval_instruct)
print(count)

## Analysing DS-1000 datapoints

In [ ]:
result, alignments = evaluate_row_constraints_strict(4, granite_2b_dfs["ds_1000"].reset_index())


In [ ]:
result, alignments = evaluate_row_constraints_strict(7, granite_2b_dfs["ds_1000"].reset_index())


In [ ]:
result, alignments = evaluate_row_constraints_strict(12, granite_2b_dfs["ds_1000"].reset_index())


In [ ]:
result, alignments = evaluate_row_constraints_strict(26, granite_2b_dfs["ds_1000"].reset_index())


In [ ]:
result, alignments = evaluate_row_constraints_strict(102, granite_2b_dfs["ds_1000"].reset_index())


In [ ]:
result, alignments = evaluate_row_constraints_strict(7, granite_2b_dfs["bigcode_bench"].reset_index())


In [ ]:
result, alignments = evaluate_row_constraints_strict(25, granite_2b_dfs["bigcode_bench"].reset_index())


In [ ]:
import re
import pandas as pd

# Pre-compile a regex that looks for:
#   "Conditions"  :  [  { ... } ( , { ... } )*  ]
# with arbitrary content inside the dicts.
_COND_PATTERN = re.compile(
    r'"Conditions"\s*:\s*\[\s*'         # opening
    r'\{'                                # first dict
      r'(?:[^{}]|\{[^{}]*\})*'           # (not perfect JSON parsing, but eats nested braces)
    r'\}'                                # close first dict
    r'(?:\s*,\s*\{(?:[^{}]|\{[^{}]*\})*\})*'  # optionally more dicts
    r'\s*\]',                            # closing ]
    re.DOTALL
)

def count_condition_structures(df: pd.DataFrame, column: str) -> int:
    """
    Scan `df[column]`, treat each entry as text, and count how many
    contain a `"Conditions": [ {...} ]` block.

    Args:
      df      : your pandas DataFrame
      column  : the name of the column (dtype object/string)

    Returns:
      int — the number of rows where that pattern was found
    """
    # .astype(str) in case there are NaNs or non-strings
    return df[df[column].astype(str).str.contains(_COND_PATTERN)]


In [ ]:
granite_2b_dfs["ds_1000"].columns

In [ ]:
filter_df = count_condition_structures(granite_2b_dfs["ds_1000"],"combined_instruction")

In [ ]:
filter_df.shape

In [ ]:
filter_df = filter_df.reset_index()

In [ ]:
print(filter_df["combined_instruction"][13])

In [ ]:
import re

# Compile once at module load
_COND_PATTERN = re.compile(
    r'"Conditions"\s*:\s*\[\s*'                    # key + opening bracket
      r'\{'                                        # first dict
        r'(?:[^{}]|\{[^{}]*\})*'                   # contents (naïve brace balancing)
      r'\}'                                        # close first dict
      r'(?:\s*,\s*\{(?:[^{}]|\{[^{}]*\})*\})*'     # optional additional dicts
    r'\s*\]',                                      # closing bracket
    re.DOTALL
)

def remove_conditions_if_present(text: str) -> str:
    """
    If `text` contains a `"Conditions": [ ... ]` block, remove that entire block;
    otherwise return it unchanged.
    """
    if _COND_PATTERN.search(text):
        # only perform the replace when the pattern is found
        return _COND_PATTERN.sub('', text)
    return text


# → 
# 'No block here.'
# 'Keep this but strip  now.'
a = granite_2b_dfs["ds_1000"]["combined_instruction"].astype(str).apply(remove_conditions_if_present)

In [ ]:
print(a.iloc[12])

In [27]:
print(granite_2b_dfs["ds_1000"]['combined_instruction'].iloc[0])

Problem:
I have the following DataFrame:
    Col1  Col2  Col3  Type
0      1     2     3     1
1      4     5     6     1
2      7     8     9     2
3    10    11    12     2
4    13    14    15     3
5    16    17    18     3

The DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, followed by the rows with Type 3, etc. I would like to shuffle the order of the DataFrame's rows according to a list. 

To ensure modularity and reusability, the function must be defined to accept a DataFrame and a list as parameters. For example, give a list [2, 4, 0, 3, 1, 5] and desired result should be:
    Col1  Col2  Col3  Type
2      7     8     9     2
4     13    14    15     3
0     1     2     3     1
3    10    11    12     2
1     4     5     6     1
5    16    17    18     3
...

The function should validate that the input list contains valid indices for the DataFrame, raising an error if any index is out of bounds. How can I achieve this

In [ ]:
len(granite_2b_dfs["ds_1000"])

In [ ]:
pattern = r'"Conditions"\s*:\s*\[\s*\{.*?\}(?:\s*,\s*\{.*?\})*\s*\]\s}'
print(granite_2b_dfs["ds_1000"].combined_instruction.str.replace(pattern, '', regex=True).str.strip().iloc[])

Problem:
I have been struggling with removing the time zone info from a column in a pandas dataframe. I have checked the following question, but it does not work for me:

Can I export pandas DataFrame to Excel stripping tzinfo?

I used tz_localize to assign a timezone to a datetime object, because I need to convert to another timezone using tz_convert. This adds an UTC offset, in the way "-06:00". I need to get rid of this offset, because it results in an error when I try to export the dataframe to Excel. 

To ensure that the output does not include timezone information when exporting to Excel, I need to remove the UTC offset from the datetime column in the pandas dataframe.

Actual output

2015-12-01 00:00:00-06:00

Desired output
2015-12-01 00:00:00

I have tried to get the characters I want using the str() method, but it seems the result of tz_localize is not a string. My solution so far is to export the dataframe to csv, read the file, and to use the str() method to get the charact

In [11]:
granite_2b_dfs["ds_1000"][granite_2b_dfs["ds_1000"].combined_instruction.str.contains(pattern)]

,id,dataset,instruction,code,test,relevant_categories,simplified_instruction,extracted_constraints,final_comprehensive_constraints,filtered_relevant_constraints,...,constraint_presence_response,final_constraints,response,constraint_adherence_responses,Constraint_adherence,correctness_level,correctness_reason,code_correctness_response,CSR_per_row,SSR_per_row
375,375,xlangai/DS-1000,Problem:\nI have the following DataFrame:\n ...,"def g(df, List):\n return df.iloc[List]\n\n...",import pandas as pd\nimport numpy as np\nimpor...,"['Code Structure and Modularity', 'Input and O...",Problem:\nI have the following DataFrame:\n ...,[],"[{'type': 'Code Structure and Modularity', 'co...","[{'type': 'Code Structure and Modularity', 'co...",...,"```json\n{\n ""Evaluation"": [\n {\n ""C...","[{'type': 'Code Structure and Modularity', 'co...",Here's a function that meets your requirements...,"```json\n{\n ""Evaluation"": [\n {\n ...","[True, True, True, True]",Completely Correct,"The code is free of syntax errors, logically m...","{""reason"": ""The code is free of syntax errors,...",1,1.000000
376,376,xlangai/DS-1000,Problem:\nI have the following DataFrame:\n ...,"def g(df, List):\n df2 = df.iloc[List].rein...",import pandas as pd\nimport numpy as np\nimpor...,"['Code Structure and Modularity', 'Input and O...",Problem:\nI have the following DataFrame:\n ...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,Here's how you can achieve this using pandas:\...,"```json\n{\n ""Evaluation"": [\n {\n ...","[True, False, False, True]",Wrong,The code has syntax errors and does not correc...,"{""reason"": ""The code has syntax errors and doe...",0,0.500000
377,377,xlangai/DS-1000,Problem:\nI have following pandas dataframe :\...,def g(df):\n return df.where(df.apply(lambd...,import pandas as pd\nimport numpy as np\nimpor...,"['Data Processing and Transformation', 'Librar...",Problem:\nI have following pandas dataframe :\...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,Here's how you can achieve this using pandas:\...,"```json\n{\n ""Evaluation"": [\n {\n ...","[False, False, False, True, False]",Wrong,The code contains a syntax error due to incorr...,"{""reason"": ""The code contains a syntax error d...",0,0.200000
378,378,xlangai/DS-1000,Problem:\nI have following pandas dataframe :\...,def g(df):\n return df.where(df.apply(lambd...,import pandas as pd\nimport numpy as np\nimpor...,"['Data Processing and Transformation', 'Librar...",Problem:\nI have following pandas dataframe :\...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,Here's the solution to your problem:\n\n```pyt...,"```json\n{\n ""Evaluation"": [\n {\n ...","[False, True, True, True, False, True]",Wrong,The code does not correctly transform the Data...,"```json\n{\n ""reason"": ""The code does not cor...",0,0.666667
379,379,xlangai/DS-1000,Problem:\nI have following pandas dataframe :\...,result = df.where(df.apply(lambda x: x.map(x.v...,import pandas as pd\nimport numpy as np\nimpor...,"['Code Structure and Modularity', 'Input and O...",Problem:\nI have following pandas dataframe :\...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,Here's the solution to your problem:\n\n```pyt...,"{""Evalu

In [14]:
print(granite_2b_dfs["ds_1000"].combined_instruction.iloc[12])

Problem:
I have been struggling with removing the time zone info from a column in a pandas dataframe. I have checked the following question, but it does not work for me:

Can I export pandas DataFrame to Excel stripping tzinfo?

I used tz_localize to assign a timezone to a datetime object, because I need to convert to another timezone using tz_convert. This adds an UTC offset, in the way "-06:00". I need to get rid of this offset, because it results in an error when I try to export the dataframe to Excel. 

To ensure that the output does not include timezone information when exporting to Excel, I need to remove the UTC offset from the datetime column in the pandas dataframe.

Actual output

2015-12-01 00:00:00-06:00

Desired output
2015-12-01 00:00:00

I have tried to get the characters I want using the str() method, but it seems the result of tz_localize is not a string. My solution so far is to export the dataframe to csv, read the file, and to use the str() method to get the charact

In [17]:
import re
pattern_identification_1 = re.compile(
    r'###\s*BEGIN\s+SOLUTION'      # the header
    r'[\s\S]*?'                     # any text, non-greedy
    r'"Conditions"\s*:\s*\[\s*\{'   # the start of the JSON‐list
    r'[\s\S]*?'                     # anything inside
    r'\}\s*\]\s}',                     # closing brace and bracket
    re.IGNORECASE                   # if you want case‐insensitive match
)
type1 = granite_2b_dfs["ds_1000"][granite_2b_dfs["ds_1000"].combined_instruction.str.contains(pattern_identification_1)]

In [26]:
pattern_replace_1 = r'"Conditions"\s*:\s*\[\s*\{.*?\}(?:\s*,\s*\{.*?\})*\s*\]\s}'
print(type1.combined_instruction.str.replace(pattern_replace_1, '', regex=True).str.strip().iloc[0])

Problem:
I have following pandas dataframe :


import pandas as pd 
from pandas import Series, DataFrame
data = DataFrame({'Qu1': ['apple', 'potato', 'cheese', 'banana', 'cheese', 'banana', 'cheese', 'potato', 'egg'],
              'Qu2': ['sausage', 'banana', 'apple', 'apple', 'apple', 'sausage', 'banana', 'banana', 'banana'],
              'Qu3': ['apple', 'potato', 'sausage', 'cheese', 'cheese', 'potato', 'cheese', 'potato', 'egg']})


I'd like to change values in columns Qu1, Qu2, Qu3 according to value_counts() when value count is greater than or equal to 2, specifically keeping values cheese, potato, and banana in column Qu1. For example for Qu1 column 
>>> pd.value_counts(data.Qu1) >= 2
cheese     True
potato     True
banana     True
apple     False
egg       False

From values apple and egg, I would like to create a value 'others' in column Qu1. For column Qu2, I want no changes:
>>> pd.value_counts(data.Qu2) >= 2
banana     True
apple      True
sausage    True

The final resul

In [28]:
pattern_identification_2 = re.compile(
    r'BEGIN\s+SOLUTION'         # BEGIN SOLUTION
    r'[\s\S]*?'                 # anything, non-greedy
    r'<code>'                   # the opening code tag
    r'[\s\S]*?'                 # anything, non-greedy
    r'"Conditions"\s*:\s*\['    # start of the Conditions array
    r'[\s\S]*?'                 # contents of the array
    r'\]'                       # closing bracket
    r'[\s\S]*?</code>',         # up through the closing code tag
    re.IGNORECASE
)
type2 = granite_2b_dfs["ds_1000"][granite_2b_dfs["ds_1000"].combined_instruction.str.contains(pattern_identification_2)]
type2

,id,dataset,instruction,code,test,relevant_categories,simplified_instruction,extracted_constraints,final_comprehensive_constraints,filtered_relevant_constraints,...,constraint_presence_response,final_constraints,response,constraint_adherence_responses,Constraint_adherence,correctness_level,correctness_reason,code_correctness_response,CSR_per_row,SSR_per_row
375,375,xlangai/DS-1000,Problem:\nI have the following DataFrame:\n ...,"def g(df, List):\n return df.iloc[List]\n\n...",import pandas as pd\nimport numpy as np\nimpor...,"['Code Structure and Modularity', 'Input and O...",Problem:\nI have the following DataFrame:\n ...,[],"[{'type': 'Code Structure and Modularity', 'co...","[{'type': 'Code Structure and Modularity', 'co...",...,"```json\n{\n ""Evaluation"": [\n {\n ""C...","[{'type': 'Code Structure and Modularity', 'co...",Here's a function that meets your requirements...,"```json\n{\n ""Evaluation"": [\n {\n ...","[True, True, True, True]",Completely Correct,"The code is free of syntax errors, logically m...","{""reason"": ""The code is free of syntax errors,...",1,1.000000
376,376,xlangai/DS-1000,Problem:\nI have the following DataFrame:\n ...,"def g(df, List):\n df2 = df.iloc[List].rein...",import pandas as pd\nimport numpy as np\nimpor...,"['Code Structure and Modularity', 'Input and O...",Problem:\nI have the following DataFrame:\n ...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,Here's how you can achieve this using pandas:\...,"```json\n{\n ""Evaluation"": [\n {\n ...","[True, False, False, True]",Wrong,The code has syntax errors and does not correc...,"{""reason"": ""The code has syntax errors and doe...",0,0.500000
377,377,xlangai/DS-1000,Problem:\nI have following pandas dataframe :\...,def g(df):\n return df.where(df.apply(lambd...,import pandas as pd\nimport numpy as np\nimpor...,"['Data Processing and Transformation', 'Librar...",Problem:\nI have following pandas dataframe :\...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,Here's how you can achieve this using pandas:\...,"```json\n{\n ""Evaluation"": [\n {\n ...","[False, False, False, True, False]",Wrong,The code contains a syntax error due to incorr...,"{""reason"": ""The code contains a syntax error d...",0,0.200000
382,382,xlangai/DS-1000,Problem:\nI have a dataset :\nid url ke...,def g(df):\n return df.loc[(df['keep_if_dup...,import pandas as pd\nimport numpy as np\nimpor...,"['Data Processing and Transformation', 'Input ...",Problem:\nI have a dataset :\nid url ke...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,"result = df.loc[df.duplicated(subset='url', ke...","```json\n{\n ""Evaluation"": [\n {\n ...","[True, False, False, True, True]",Wrong,The code does not correctly implement the logi...,"{""reason"": ""The code does not correctly implem...",0,0.600000
383,383,xlangai/DS-1000,Problem:\nI have a dataset :\nid url dr...,def g(df):\n return df.loc[(df['drop_if_dup...,import pandas as pd\nimport numpy as np\nimpor...,"['Data Processing and Transformation', 'Input ...",Problem:\nI have a dataset :\nid url dr...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,[{'type': 'Data Processing and Transformation'...,...,"```json\n{\n ""Evaluation"": [\n {\n ""C...",[{'type': 'Data Processing and Transformation'...,"result = df.loc[df.duplicated(subset='url', ke...","```json\n{\n ""Evaluation"": [\n {\n ...

In [ ]:
pattern_identification_3 = re.compile(
    r'BEGIN\s+SOLUTION'       # the header
    r'[\s\S]*?'                     # any text, non-greedy
    r'"Conditions"\s*:\s*\[\s*\{'   # the start of the JSON‐list
    r'[\s\S]*?'                     # anything inside
    r'\}\s*\]\s}',                     # closing brace and bracket
    re.IGNORECASE                   # if you want case‐insensitive match
)
type3 = granite_2b_dfs["ds_1000"][granite_2b_dfs["ds_1000"].combined_instruction.str.contains(pattern_identification_3)]
type3

,id,dataset,instruction,code,test,relevant_categories,simplified_instruction,extracted_constraints,final_comprehensive_constraints,filtered_relevant_constraints,...,constraint_presence_response,final_constraints,response,constraint_adherence_responses,Constraint_adherence,correctness_level,correctness_reason,code_correctness_response,CSR_per_row,SSR_per_row


# V8 Dataset 

In [48]:
import pandas as pd 
df = pd.read_json("/dccstor/shanmukh/sravani_internship/benchmark_experiments/LLMjudge_outputs/code_correctness/deepseek-coder-33b-instruct_results_correctness.jsonl",lines=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

# Prompt for constarints easy and hard classification

In [7]:
def constraint_difficulty_classification(instruction, constraints):
    prompt = f""" 
    You are an assistant that classifies requirement-constraints into “easy” vs. “hard” based on their implementation complexity, performance sensitivity, and testing burden. 

• Easy constraints involve only single-step logic, simple built-in calls, or trivial checks (O(1) or O(N) over very small N), have minimal performance impact, and require almost no special testing.  
• Hard constraints include anything that goes beyond trivial checks: multi-stage algorithms, strict resource bounds (e.g. O(1) space, custom prime testing), unusual data types or math, high performance or security sensitivity, or require extensive edge-case testing.  

JSON Response Format:
{{
  "Classification": [
    {{
      "Constraint": "<constraint text> (just the text inside quotes, no escape characters)",
      "Reason": "<brief explanation>",
      "label": "hard" | "easy"
    }},
    ...
  ]
}}

Strictly adhere to valid JSON. 
[Instruction]:
{instruction}

[Constraints]:
{constraints}
    """
    return prompt

In [8]:
def extract_label_values(text):
    """
    Extracts all occurrences of the "label" field that are either "Easy" or "Hard"
    from a JSON-like string block. Returns a list of label strings.
    """
    # Only match labels "Easy" or "Hard"
    matches = re.findall(r'"label"\s*:\s*"(easy|hard)"', text, re.IGNORECASE)
    return matches

In [55]:
n = 100
id =  5
combined_instruction = df[df["id"]==id]["combined_instruction"].values[0]
constraints = df[df["id"]==id]["final_constraints"].values[0]
print(combined_instruction)
print(constraints)
prompt = constraint_difficulty_classification(combined_instruction,constraints)
response = client.get_model_response(user_prompt=prompt)
print("Classification:\n",response)

Create an array of length N (where N is a positive integer) containing numbers divisible by M (where M is a positive integer) up to X (where X is a positive integer). Each number in the array must be unique and in ascending order, as required. Additionally, the sum of all the numbers in the array should be a prime number, which must be computed without using any built-in functions or libraries to check for prime numbers. The solution should be optimized to find the largest prime sum within the given constraints. Furthermore, the time complexity of the solution should be O(N) and the space complexity should be O(1). The solution should handle cases where no prime number can be obtained as the sum of the array elements; in such cases, the solution should return an empty array. It should also handle cases where multiple prime numbers can be obtained as the sum of the array elements, returning the array with the largest sum that is prime. Additionally, the solution should minimize the numb

Classification:
 ```json
{
  "Classification": [
    {
      "Constraint": "Each number in the array must be unique and in ascending order.",
      "Reason": "This involves simple checks and ordering, which can be done in a single pass with straightforward logic and minimal complexity.",
      "label": "Easy"
    },
    {
      "Constraint": "The sum of all the numbers in the array should be a prime number.",
      "Reason": "Checking primality, especially without built-in functions, requires implementing a prime test algorithm which can be non-trivial depending on input size and optimization needs.",
      "label": "Hard"
    },
    {
      "Constraint": "The time complexity of the solution should be O(N).",
      "Reason": "Ensuring O(N) time complexity requires careful algorithm design and optimization, especially when combined with prime checking and sum calculations.",
      "label": "Hard"
    },
    {
      "Constraint": "The space complexity of the solution should be O(1).",
  

In [56]:
labels = extract_label_values(response)
labels

['Easy', 'Hard', 'Hard', 'Hard', 'Hard', 'Easy', 'Hard', 'Hard', 'Hard']

# Filtering the Benchmark_v9 dataset 

In [103]:
import re

def extract_difficulty_from_block(text: str) -> str:
    """
    Extracts the value of "Difficulty" from a JSON-like string block.
    Returns the difficulty (e.g. "Easy", "Medium", "Hard") or None if not found.
    """
    matches = re.findall(
        r'"Difficulty"\s*:\s*"(Easy|Medium|Hard|easy|medium|hard)"',
        text,
        flags=re.IGNORECASE
    )
    return [m.lower() for m in matches][0] if matches else None


# Example usage:
s = "```json\n{\n  \"Reason\": \"The task requires detecting a cycle in a linked list and returning the node where the cycle begins, using the two-pointer technique with O(1) memory. This involves understanding Floyd's cycle detection algorithm and careful pointer manipulation, as well as handling edge cases and invalid inputs. While the algorithm is standard, it requires moderate understanding of linked lists and pointer logic.\",\n  \"Difficulty\": \"easy\"\n}\n```"
print(extract_difficulty_from_block(s))  # Output: medium


easy


In [108]:
import pandas as pd
df = pd.read_json("benchmark_dataset/benchmark_v10.jsonl", lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [104]:
df["difficulty"] = df["difficulty_response"].apply(lambda x: extract_difficulty_from_block(x))

In [107]:
df.to_json("benchmark_dataset/benchmark_v10.jsonl", orient='records', lines=True)

In [58]:
df["difficulty"].value_counts()

difficulty
medium    952
hard      324
easy      161
Name: count, dtype: int64

In [59]:
df_2 = pd.read_json("benchmark_dataset/benchmark_v10_v2.jsonl", lines=True)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               100 non-null    int64  
 1   dataset                          100 non-null    object 
 2   instruction                      100 non-null    object 
 3   code                             100 non-null    object 
 4   test                             0 non-null      float64
 5   relevant_categories              100 non-null    object 
 6   simplified_instruction           100 non-null    object 
 7   extracted_constraints            100 non-null    object 
 8   final_comprehensive_constraints  100 non-null    object 
 9   filtered_relevant_constraints    100 non-null    object 
 10  quality_scores                   100 non-null    object 
 11  relevance_score                  100 non-null    float64
 12  objectivity_score      

In [81]:
df["difficulty"] = df["difficulty_response"].apply(lambda x: extract_difficulty_from_block(x))

In [82]:
df[df["difficulty"].isna()]["difficulty_response"].value_counts()

Series([], Name: count, dtype: int64)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [84]:
df.to_json("benchmark_dataset/benchmark_v10.jsonl", orient="records", lines=True)

In [85]:
df = pd.read_json("benchmark_dataset/benchmark_v10.jsonl", lines=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [24]:
import pandas as pd
df_2 = pd.read_json("benchmark_dataset/benchmark_v10_v2.jsonl", lines=True)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               1489 non-null   int64  
 1   dataset                          1489 non-null   object 
 2   instruction                      1489 non-null   object 
 3   code                             1489 non-null   object 
 4   test                             590 non-null    object 
 5   relevant_categories              1489 non-null   object 
 6   simplified_instruction           1489 non-null   object 
 7   extracted_constraints            1489 non-null   object 
 8   final_comprehensive_constraints  1489 non-null   object 
 9   filtered_relevant_constraints    1489 non-null   object 
 10  quality_scores                   1489 non-null   object 
 11  relevance_score                  1489 non-null   float64
 12  objectivity_score   

In [102]:
df_2.head()

,id,dataset,instruction,code,test,relevant_categories,simplified_instruction,extracted_constraints,final_comprehensive_constraints,filtered_relevant_constraints,...,atomicity_score,unified_quality_score,combined_instruction,constraint_wise_presence,constraint_presence_response,final_constraints,difficulty,difficulty_response,constraint_difficulty_labels,constraint_difficulty_response
0,1400,Multilingual-Multimodal-NLP/McEval-Instruct,### 142. Linked List Cycle II\n\nGiven the `he...,```python\nfrom ListNode import *\n\n# Definit...,NaN,"['Code Structure and Modularity', 'Input and O...","Given the `head` of a linked list, return the ...","[{'type': 'Input and Output Handling', 'constr...","[{'type': 'Input and Output Handling', 'constr...","[{'type': 'Input and Output Handling', 'constr...",...,4.60,4.60,### 142. Linked List Cycle II\n\nGiven the `he...,"[True, True, True, True, True, True, True, Tru...","{\n ""Evaluation"": [\n {\n ""Constraint...","[{'type': 'Input and Output Handling', 'constr...",[medium],"```json\n{\n ""Reason"": ""The task involves det...","[easy, easy, easy, easy, easy, hard, hard, har...","```json\n{\n ""Classification"": [\n {\n ..."
1,1401,Multilingual-Multimodal-NLP/McEval-Instruct,Design a Django database migration that create...,```python\n# -*- coding: utf-8 -*-\nfrom __fut...,NaN,"['Code Structure and Modularity', 'Input and O...",Design a Django database migration that create...,"[{'type': 'Code Structure and Modularity', 'co...","[{'type': 'Code Structure and Modularity', 'co...","[{'type': 'Code Structure and Modularity', 'co...",...,5.00,5.00,Design a Django database migration that create...,"[True, True, True, True, True, True, True, Tru...","{\n ""Evaluation"": [\n {\n ""Constraint...","[{'type': 'Code Structure and Modularity', 'co...",[medium],"```json\n{\n ""Reason"": ""The task involves cre...","[easy, easy, easy, easy, easy, easy, easy, eas...","{\n ""Classification"": [\n {\n ""Constr..."
2,1402,Multilingual-Multimodal-NLP/McEval-Instruct,Implement a class `PoseNet` that uses a Tensor...,```python\nimport tflite_runtime.interpreter a...,NaN,"['Code Structure and Modularity', 'Input and O...",Implement a class `PoseNet` that uses a Tensor...,"[{'type': 'Library and API Usage', 'constraint...","[{'type': 'Library and API Usage', 'constraint...","[{'type': 'Library and API Usage', 'constraint...",...,5.00,5.00,Implement a class `PoseNet` that uses a Tensor...,"[True, True, True, True, True, True, True, Tru...","{\n ""Evaluation"": [\n {\n ""Constraint...","[{'type': 'Library and API Usage', 'constraint...",[hard],"```json\n{\n ""Reason"": ""The task involves mul...","[easy, easy, easy, hard, easy, easy, easy, har...","{\n ""Classification"": [\n {\n ""Constr..."
3,1403,Multilingual-Multimodal-NLP/McEval-Instruct,Create a Python script using Django's admin fu...,```python\nfrom django.contrib import admin\nf...,NaN,"['Code Structure and Modularity', 'Input and O...",Create a Python script using Django's admin fu...,"[{'type': 'Code Structure and Modularity', 'co...","[{'type': 'Code Structure and Modularity', 'co...","[{'type': 'Code Structure and Modularity', 'co...",...,4.85,4.83,Create a Python script using Django's admin fu...,"[True, True, True, True, True, True, True, Tru...","{\n ""Evaluation"": [\n {\n ""Constraint...","[{'type': 'Code Structure and Modularity', 'co...",[medium],"```json\n{\n ""Reason"": ""The task involves cre...","[easy, easy, easy, easy, easy, easy, easy, eas...","```json\n{\n ""Classification"": [\n {\n ..."
4,1404,Multilingual-Multimodal-NLP/McEval-Instruct,Write a Python program that automates the proc...,```python\nimport json\nimport requests\nfrom ...,NaN,"['Code Structure and Modularity', 'Input and O...",Write a Python program that automates the proc...,"[{'type': 'File and Data Management', 'constra...","[{'type': 'File and Data Management', 'constra...","[{'type': 'File and Data Management', 'constra...",...,5.00,5.00,Write a Python program that auto

In [14]:
import os
import dotenv
dotenv.load_dotenv()
from utils.compute_clients import create_clients
api_key = os.getenv("IBM_OPENAI_API_KEY")
client = create_clients(mode="GPT-azure")

In [17]:
import re
for i in range(len(df_2)):
    if len(df_2["final_constraints"][i]) != len(df_2["constraint_difficulty_labels"][i]):
        print(f"Row {i} has mismatched lengths: {len(df_2['final_constraints'][i])} constraints vs {len(df_2['constraint_difficulty_labels'][i])} difficulties")
        print("Constraints:", df_2["final_constraints"][i])
        print("Difficulties:", df_2["constraint_difficulty_labels"][i])
        print("response_outputs",df_2["constraint_difficulty_response"][i])
        # regenrate the response for this row
        # prompt = constraint_difficulty_classification(
        #     df_2["combined_instruction"][i],
        #     df_2["final_constraints"][i]
        # )
        # df_2["constraint_difficulty_response"][i] = client.get_model_response(user_prompt=prompt)
        # df_2["constraint_difficulty_labels"][i] = extract_label_values(df_2["constraint_difficulty_response"][i])
        # print("Updated response:", df_2["constraint_difficulty_response"][i])
        # print("Updated labels:", df_2["constraint_difficulty_labels"][i])
        # print(len(df_2["final_constraints"][i]) == len(df_2["constraint_difficulty_labels"][i]))
        print("-" * 40)
  

In [20]:
num_hard = df_2["constraint_difficulty_labels"].apply(lambda x: x.count("hard"))
num_easy = df_2["constraint_difficulty_labels"].apply(lambda x: x.count("easy"))
print("Total Hard Constraints:", num_hard.sum())
print("Total Easy Constraints:", num_easy.sum())
print("Total Constraints:", (num_hard + num_easy).sum())
print("total_constraints:",len(df_2["final_constraints"].sum()))

Total Hard Constraints: 3270
Total Easy Constraints: 7204
Total Constraints: 10474
total_constraints: 10474


In [22]:
df_2 = df_2.rename(columns={
    "difficulty": "instruction_difficulty_labels"})


In [23]:
df_2.to_json("benchmark_dataset/benchmark_v10_v2.jsonl", orient='records', lines=True)

In [51]:
df_2["instruction_difficulty_labels"].value_counts()

instruction_difficulty_labels
medium    1001
hard       324
easy       164
Name: count, dtype: int64

In [33]:
# now I have one df_2 and a folder of dfs , now read every jsonl file from the folder convert it to df and then i want to add 2 columns [instruction_difficulty_labels,constraint_difficulty_labels] from df_2 to each of the dfs in the folder by matching their values in the id column if the ids in df_2 not present in the corresponding df then continue and then save them back to the folder with the same name
import os
import json
import pandas as pd 
df_2 = pd.read_json("benchmark_dataset/benchmark_v10_v2.jsonl", lines=True)
# folder_path = "LLMjudge_outputs/filtered_final_data/"
# folder_path = "LLMjudge_outputs/code_correctness/"
folder_path = "metrics_outputs/filtered_final_data/"
for filename in os.listdir(folder_path):
    if filename=="metrics_summary_v4.jsonl":
        continue
    if filename.endswith(".jsonl"):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        print(f"File path: {file_path}")
        df = pd.read_json(file_path, lines=True)
        print("no of columns in df:", len(df.columns))
        # Merge df with df_2 on 'id' column
        # check if the new columns exist in the  DataFrame
        if "instruction_difficulty_labels" in df.columns and "constraint_difficulty_labels" in df.columns:
            print(f"Columns already exist in {filename}, skipping merge.")
            continue
        merged_df = df.merge(df_2[["id", "instruction_difficulty_labels", "constraint_difficulty_labels"]],
                             on="id", how="left")
        # Save the merged DataFrame back to the same file
        print(f"Merging {filename} with df_2...")
        print(f"Original shape: {df.shape}, Merged shape: {merged_df.shape}")
        # check for any NaN values in the new columns
       
        if merged_df["instruction_difficulty_labels"].isna().any() or merged_df["constraint_difficulty_labels"].isna().any():
            print("Warning: Some rows have NaN values in the new columns.")
        # Save the merged DataFrame back to the file
        merged_df.to_json(file_path, orient='records', lines=True)



Processing file: granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl
File path: metrics_outputs/filtered_final_data/granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl
no of columns in df: 28
Columns already exist in granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl, skipping merge.
Processing file: mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl
File path: metrics_outputs/filtered_final_data/mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl
no of columns in df: 28
Columns already exist in mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl, skipping merge.
Processing file: Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl
File path: metrics_outputs/filtered_final_data/Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl
no of columns in df: 28
Columns already exist in Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl, skipping merge.
Proc

In [44]:
# now take a folder of jsonl files and calculate the average ssr and the average CSR for each value in instruction_difficulty_labels and also average ssr for each value in constraint_difficulty_labels like ssr for easy constraints and ssr for hard constraints
import os
import pandas as pd
folder_path = "metrics_outputs/filtered_final_data/"
result_df = []
for filename in os.listdir(folder_path):
    if filename == "metrics_summary_v4.jsonl" :
        continue
    if filename.endswith(".jsonl"):
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        df = pd.read_json(file_path, lines=True)
        # Calculate average SSR for each value in instruction_difficulty_labels
        avg_ssr_instruction = df.groupby("instruction_difficulty_labels")["SSR_per_row"].mean()
        
        print(f"Average SSR by Instruction Difficulty for {filename}:\n{avg_ssr_instruction}\n")
        
        # Calculate average CSR for each value in instruction_difficulty_labels
        avg_csr_instruction = df.groupby("instruction_difficulty_labels")["CSR_per_row"].mean()
        print(f"Average CSR by Instruction Difficulty for {filename}:\n{avg_csr_instruction}\n")
        
        # Save the results to a new file
        result_df.append({
            "filename": filename,
            "avg_ssr_instruction": avg_ssr_instruction.to_dict(),
            "avg_csr_instruction": avg_csr_instruction.to_dict(),
        })
result_file_path = os.path.join(folder_path, f"results_easy_hard_instruction.jsonl")
pd.DataFrame(result_df).to_json(result_file_path, orient='records', lines=True)
print("results saved to:", result_file_path)


Processing file: granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl


Average SSR by Instruction Difficulty for granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl:
instruction_difficulty_labels
easy      0.869271
hard      0.772841
medium    0.841583
Name: SSR_per_row, dtype: float64

Average CSR by Instruction Difficulty for granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl:
instruction_difficulty_labels
easy      0.590909
hard      0.247387
medium    0.437403
Name: CSR_per_row, dtype: float64

Processing file: mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl
Average SSR by Instruction Difficulty for mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl:
instruction_difficulty_labels
easy      0.554584
hard      0.665187
medium    0.665533
Name: SSR_per_row, dtype: float64

Average CSR by Instruction Difficulty for mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl:
instruction_difficulty_labels
easy      0.287879
hard      0.139373
medium    0.241113
Name: CSR_per_

KeyError: 'instruction_difficulty_labels'

In [42]:
print(avg_ssr_instruction.to_dict())

{'easy': 0.8453692771833333, 'hard': 0.7343230085836236, 'medium': 0.8673727354545595}


In [50]:
result_df = pd.read_json(result_file_path, lines=True)
result_df.info()
# now check if the trend is not holding easy>medium>hard for both SSR and CSR, if not then print the filename and the values
for filename, row in result_df.iterrows():
    avg_ssr = row["avg_ssr_instruction"]
    avg_csr = row["avg_csr_instruction"]
    
    # Check if the trend is not holding
    if not (avg_ssr.get("easy", 0) >= avg_ssr.get("medium", 0) >= avg_ssr.get("hard", 0)):
        print(f"SSR trend not holding in {row['filename']}: {avg_ssr}")

    if not (avg_csr.get("easy", 0) >= avg_csr.get("medium", 0) >= avg_csr.get("hard", 0)):
        print(f"CSR trend not holding in {row['filename']}: {avg_csr}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   filename             11 non-null     object
 1   avg_ssr_instruction  11 non-null     object
 2   avg_csr_instruction  11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes
SSR trend not holding in mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl: {'easy': 0.5545843046, 'hard': 0.6651870584, 'medium': 0.6655326384}
SSR trend not holding in Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl: {'easy': 0.8541264291, 'hard': 0.7610444192, 'medium': 0.8608996962000001}
SSR trend not holding in granite-3.1-8b-instruct_results_correctness_metrics_extended.jsonl: {'easy': 0.814585793, 'hard': 0.7506612532, 'medium': 0.8216819481000001}
SSR trend not holding in Qwen3-8B_results_correctness_metrics_extended.jsonl: {'easy': 0.9077023986, 'hard': 0

In [ ]:
import os
import pandas as pd
from collections import Counter

def compute_ssr_by_difficulty(constraint_labels, constraint_adherence):
    counts = Counter({"easy": 0, "hard": 0})
    trues  = Counter({"easy": 0, "hard": 0})

    for lbl, ok in zip(constraint_labels, constraint_adherence):
        lbl = lbl.lower()
        if lbl in counts:
            counts[lbl] += 1
            if ok:
                trues[lbl] += 1

    return {
        "easy":  trues["easy"]  / counts["easy"]  if counts["easy"]  > 0 else None,
        "hard":  trues["hard"]  / counts["hard"]  if counts["hard"]  > 0 else None,
    }

def summarize_ssr_for_folder(folder_path):
    """
    Reads each JSONL file in folder_path (already merged with labels & adherence),
    computes SSR for easy vs. hard constraints, and returns a DataFrame:
      filename | ssr_easy | ssr_hard
    """
    records = []
    for fname in os.listdir(folder_path):
        if fname == "metrics_summary_v4.jsonl" or fname == "results_easy_hard_instruction.jsonl":
            continue
        if not fname.endswith(".jsonl"):
            continue
        path = os.path.join(folder_path, fname)
        print(f"Processing file: {fname}")
        df = pd.read_json(path, lines=True)

        # flatten all rows' lists into two flat lists
        all_labels    = [lbl for row in df["constraint_difficulty_labels"] for lbl in row]
        all_adherence = [ok  for row in df["Constraint_adherence"] for ok  in row]

        ssr = compute_ssr_by_difficulty(all_labels, all_adherence)
        records.append({
            "filename": fname,
            "ssr_easy": ssr["easy"],
            "ssr_hard": ssr["hard"],
        })
        print(f"Computed SSR for {fname}: easy={ssr['easy']}, hard={ssr['hard']}")

    return pd.DataFrame(records)

# Usage
folder_path = "metrics_outputs/filtered_final_data/"
results_df = summarize_ssr_for_folder(folder_path)
print(results_df)


Processing file: granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl
Computed SSR for granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl: easy=0.8470223599734337, hard=0.7942210074189769
Processing file: mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl


Computed SSR for mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl: easy=0.6807615674119991, hard=0.6427176884029676
Processing file: Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl
Computed SSR for Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl: easy=0.8598627407571396, hard=0.8024209293244826
Processing file: Mistral-Small-3.1-24B-Instruct-2503_results_correctness_metrics_extended.jsonl
Computed SSR for Mistral-Small-3.1-24B-Instruct-2503_results_correctness_metrics_extended.jsonl: easy=0.924507416426832, hard=0.8789535337758688
Processing file: granite-3.1-8b-instruct_results_correctness_metrics_extended.jsonl
Computed SSR for granite-3.1-8b-instruct_results_correctness_metrics_extended.jsonl: easy=0.8293114899269427, hard=0.7746973838344396
Processing file: Qwen3-8B_results_correctness_metrics_extended.jsonl
Computed SSR for Qwen3-8B_results_correctness_metrics_extended.jsonl: easy=0.9280655157149181, hard=0.8852010933229207
Pr

In [60]:
results_df.to_json(os.path.join(folder_path, "Constraint_easy_hard_ssr.jsonl"), orient='records', lines=True)

In [61]:
# check if the trend is not holding easy>hard for  SSR , if not then print the filename and the values
for _, row in results_df.iterrows():
    ssr_easy = row["ssr_easy"]
    ssr_hard = row["ssr_hard"]
    
    # Check if the trend is not holding
    if ssr_easy is None or ssr_hard is None or not (ssr_easy >= ssr_hard):
        print(f"SSR trend not holding in {row['filename']}: easy={ssr_easy}, hard={ssr_hard}")

# Compute metrics for easy and hard and medium difficulties of the instruction

In [5]:
import os
import json
import pandas as pd
import ast

# Path to summary file and folder with detailed JSONLs
folder_path = "metrics_outputs/filtered_final_data/"
summary_file_path = os.path.join(folder_path, "metrics_summary_v4.jsonl")

# Load existing summary
with open(summary_file_path, "r") as f:
    summaries = [json.loads(line) for line in f]

# Index summary entries by filename
summary_dict = {entry["filename"]: entry for entry in summaries}

# Process each *_metrics_extended.jsonl file
for filename in os.listdir(folder_path):
    if not filename.endswith("_metrics_extended.jsonl"):
        continue

    # Convert to match the filename key in summary
    base_filename = filename.replace("_metrics_extended", "")

    if base_filename not in summary_dict:
        continue

    file_path = os.path.join(folder_path, filename)
    df = pd.read_json(file_path, lines=True)

    # # Parse constraints and adherence lists
    # df["final_constraints"] = df["final_constraints"].apply(
    #     lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    # )
    # df["constraint_adherence_list"] = df["constraint_adherence_list"].apply(
    #     lambda x: list(map(int, ast.literal_eval(x))) if isinstance(x, str) else x
    # )

    # # Compute CSR/SSR per row
    # df["CSR_per_row"] = df["constraint_adherence_list"].apply(lambda lst: int(all(lst)))
    # df["SSR_per_row"] = df["constraint_adherence_list"].apply(lambda lst: sum(lst)/len(lst) if lst else 0)

    # Compute CSR & SSR by instruction difficulty
    if "instruction_difficulty_labels" in df.columns:
        avg_csr = df.groupby("instruction_difficulty_labels")["CSR_per_row"].mean().to_dict()
        avg_ssr = df.groupby("instruction_difficulty_labels")["SSR_per_row"].mean().to_dict()
    else:
        avg_csr = {}
        avg_ssr = {}

    # Update the summary entry
    summary_dict[base_filename]["avg_csr_by_instruction_difficulty"] = avg_csr
    summary_dict[base_filename]["avg_ssr_by_instruction_difficulty"] = avg_ssr

# Save updated summaries back
with open(summary_file_path, "w") as f:
    for entry in summary_dict.values():
        f.write(json.dumps(entry) + "\n")

print("✅ Updated summary file with avg CSR and SSR by instruction difficulty.")


✅ Updated summary file with avg CSR and SSR by instruction difficulty.


In [14]:
import os
import pandas as pd
import ast

# Folder path containing *_metrics_extended.jsonl files
folder_path = "metrics_outputs/filtered_final_data/"

# Initialize storage for results
csr_data = []
ssr_data = []

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith("_metrics_extended.jsonl"):
        model_name = filename.replace("_metrics_extended.jsonl", "")
        file_path = os.path.join(folder_path, filename)
        df = pd.read_json(file_path, lines=True)
        
        # # Ensure final_constraints is parsed
        # df["final_constraints"] = df["final_constraints"].apply(
        #     lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        # )
        
        # # Calculate number of constraints per record
        # df["num_constraints"] = df["final_constraints"].apply(len)
        
        # Group by number of constraints and compute mean CSR, SSR, and count
        grouped = df.groupby("num_constraints").agg({
            "CSR_per_row": "mean",
            "SSR_per_row": "mean",
            "final_constraints": "count"  # This gives count of rows per group
        }).rename(columns={"final_constraints": "count"}).reset_index()
        
        grouped["model"] = model_name

        csr_data.append(grouped[["num_constraints", "CSR_per_row", "count", "model"]])
        ssr_data.append(grouped[["num_constraints", "SSR_per_row", "count", "model"]])

# Combine data
csr_df = pd.concat(csr_data)
ssr_df = pd.concat(ssr_data)

# Save to disk
csr_df.to_json(os.path.join(folder_path, "csr_data_num_constraints.jsonl"), orient="records", lines=True)
ssr_df.to_json(os.path.join(folder_path, "ssr_data_num_constraints.jsonl"), orient="records", lines=True)

# Debug print
print(csr_df.info())
print(ssr_df.info())
print(csr_df.head())


<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 0 to 17
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   num_constraints  252 non-null    int64  
 1   CSR_per_row      252 non-null    float64
 2   count            252 non-null    int64  
 3   model            252 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 17.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 0 to 17
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   num_constraints  252 non-null    int64  
 1   SSR_per_row      252 non-null    float64
 2   count            252 non-null    int64  
 3   model            252 non-null    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 17.9+ KB
None
   num_constraints  CSR_per_row  count  \
0                1     0.750000      4   
1                2     0.631579 

In [13]:
import os
import ast
import pandas as pd

def add_num_constraints_column(folder_path):
    for filename in os.listdir(folder_path):
        if "results_correctness" in filename and filename.endswith(".jsonl"):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing: {filename}")

            try:
                df = pd.read_json(file_path, lines=True)

                # Ensure final_constraints is a list
                df["final_constraints"] = df["final_constraints"].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
                )

                # Add num_constraints column
                df["num_constraints"] = df["final_constraints"].apply(len)

                # Save back to the same file
                df.to_json(file_path, orient="records", lines=True, force_ascii=False)
                print(f"✅ Updated and saved: {filename}")
            except Exception as e:
                print(f"❌ Failed to process {filename}: {e}")

# Example usage
if __name__ == "__main__":
    folder = "metrics_outputs/filtered_final_data/"  # Replace with your actual folder path
    add_num_constraints_column(folder)


Processing: granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl
✅ Updated and saved: granite-3.3-8b-instruct_results_correctness_metrics_extended.jsonl
Processing: mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl
✅ Updated and saved: mixtral-8x7B-instruct-v0.1_results_correctness_metrics_extended.jsonl
Processing: Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl
✅ Updated and saved: Llama-3.1-8B-Instruct_results_correctness_metrics_extended.jsonl
Processing: Mistral-Small-3.1-24B-Instruct-2503_results_correctness_metrics_extended.jsonl
✅ Updated and saved: Mistral-Small-3.1-24B-Instruct-2503_results_correctness_metrics_extended.jsonl
Processing: granite-3.1-8b-instruct_results_correctness_metrics_extended.jsonl
✅ Updated and saved: granite-3.1-8b-instruct_results_correctness_metrics_extended.jsonl
Processing: Qwen3-8B_results_correctness_metrics_extended.jsonl
✅ Updated and saved: Qwen3-8B_results_correctness_metrics_extended.jsonl
Pr

In [18]:
import pandas as pd
import ast

# Load your dataset
df = pd.read_json("metrics_outputs/filtered_final_data/granite-3.1-8b-instruct_results_correctness_metrics_extended.jsonl", lines=True)  # update path if needed

def count_constraints(row):
    extracted = 0
    new = 0
    try:
        constraints = ast.literal_eval(row) if isinstance(row, str) else row
        for c in constraints:
            if c["instruction_part"] == "Extracted from instruction":
                extracted += 1
            elif c["instruction_part"] == "Newly Generated":
                new += 1
    except:
        pass
    return pd.Series([extracted, new, extracted + new])

# Apply constraint counting logic
df[["Extracted Constraints", "New Constraints", "Total Constraints"]] = df["final_constraints"].apply(count_constraints)

# Compute descriptive stats
columns = ["Extracted Constraints", "New Constraints", "Total Constraints"]
summary = df[columns].describe(percentiles=[0.25, 0.5, 0.75]).loc[["mean", "min", "25%", "50%", "75%", "max"]]
summary.index = ["Mean", "Min", "25%", "50% (Median)", "75%", "Max"]
summary = summary.T.round(2)

# Display as table
print(summary)

                       Mean  Min  25%  50% (Median)  75%   Max
Extracted Constraints  4.61  0.0  3.0           4.0  6.0  18.0
New Constraints        2.43  0.0  1.0           2.0  4.0   9.0
Total Constraints      7.04  0.0  5.0           7.0  9.0  18.0


In [16]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 139.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 168.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 128.8 MB/s eta 0:00:00
ERROR: Could not install packages due to an OSError: [Errno 122] Disk quota exceeded

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/6 [pyparsing]
